## Data Labeling 

### Goal
The goal of this project is the create an unsupervised learning algortithim that splits participants into political subgroups based on this data

#### By using the Categorical Encoder the work here is essentially useless 

In [1910]:

import pandas as pd 
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pyreadstat
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib

In [1911]:
df, meta = pyreadstat.read_sav("./Sept18/Sept18_public.sav")

In [1912]:
pd.set_option('display.max_columns', 500)


In [1926]:
nat_df = pd.read_csv("./natsurvey/national_survey.csv")

/Users/nathan/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (73,119) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [1896]:
nat_df.shape

(19221, 415)

In [1929]:
nat_df.replace(r'\s+|^$', np.nan, regex=True).head(2)

,ID,HOU,SEN,GOV,ABORTION18,AGE,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BOYSGIRLS,BREAK18,BREAK18A,BREAK18B,BRNAGAIN,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3
0,25480,NaN,NaN,NaN,NaN,30-44,30-59,30-39,18-44,18-49,30-44,30-39,NaN,NaN,NaN,NaN,NaN,NaN,Complete,NaN,NaN,NaN,NaN,NaN,NaN,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Favorable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,Approve,NaN,NaN,18-44,18-64,NaN,Democrat,NaN,NaN,NaN,Democrat,NaN,NaN,Moderate,NaN,NaN,NaN,White,White,NaN,NaN,NaN,White,NaN,Important,White,White,NaN,West,Catholic,Catholic,Catholic,Justified,Female,NaN,NaN,NaN,NaN,NaN,NaN,English,California,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.15386927127838,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,25481,NaN,NaN,NaN,NaN,30-44,30-59,30-39,18-44,18-49,30-44,30-39,NaN,NaN,NaN,NaN,NaN,NaN,Complete,NaN,NaN,NaN,NaN,NaN,NaN,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non-whites,NaN,NaN,NaN,Favorable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,Approve,NaN,NaN,18-44,18-64,NaN,Democrat,NaN,NaN,NaN,Democrat,NaN,NaN,Liberal,NaN,NaN,NaN,Other,Hispanic/Latino,NaN,NaN,NaN,Hispanic/Latino,NaN,Important,Non-white,Hispanic/Latino,NaN,West,Catholic,Catholic,NaN,Justified,Male,NaN,NaN,NaN,NaN,Suburbs,Suburbs,English,California,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.15388107299805,Non-white,NaN,NaN,NaN,NaN,NaN,NaN


In [1930]:
nat_df.shape

(19221, 127)

### Data Cleaning 
In order to process the survey data into an unsupervised clustering algorithim, the data needs to be converted into numeric data types. There are a few options for conversion. First, labelEncoder simply takes the values and encodes them starting with 0 and up to n-1 classes. This creates a pseudo rank, yet this can be problematic for data such as race and age. Second, is to create dummy variables from the non-rankable features. The features will be evaluated on a case by case basis. 

In [1931]:
def dummy_maker(string, nat_df):
    nat_df = pd.get_dummies(nat_df, columns =[string], drop_first= True)
    return nat_df

In [1932]:
def label_converter(nat_df, string): 
    labelencoder = LabelEncoder()
    labelencoder.fit(nat_df[string])
    nat_df[string] = labelencoder.transform(nat_df[string])
    return nat_df

### Question: In today's election for U.S. House of Representatives, did you just vote for:
1. The Democratic candidate
2. The Republican candidate
3. Other: Who?
4. Did not vote

In [1933]:
nat_df['HOU'].value_counts()


The Democratic candidate    11535
The Republican candidate     6934
                              319
Other                         309
Did not vote                  124
Name: HOU, dtype: int64

In [1936]:
label_converter(nat_df, 'HOU').head(2)

,ID,HOU,SEN,GOV,ABORTION18,AGE,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BOYSGIRLS,BREAK18,BREAK18A,BREAK18B,BRNAGAIN,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3
0,25480,4,The Democratic candidate,The Republican candidate,,30-44,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,,,,Complete,,,,,,,23,,,,,An advanced degree after a bachelor's degree (...,College graduate,More than high school graduate,White college degree,White women college graduates,,,,Favorable,,,,,,,Minor changes,,Yes,,,,,,,Gun policy,,No,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,White Democrat,Democratic women,Democrat,Democratic Women,White Democrat,Moderate,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,White,White,,,National questionnaire,White,Very important,Important,White,White,Whites are favored over minorities,West,Catholic,Catholic,Catholic,Justified,Female,White women,,,,"Cities 50,000-500,000","Cities over 50,000",English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,1.15386927127838,White Catholics,,,,,,
1,25481,3,The Democratic candidate,The Democratic candidate,,30-44,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,Latino 30-44,Latino 30-44,,,,Complete,,,,,,,40,,,,,Attended college but received no degree,No college degree,More than high school graduate,Non white no college degree,Non-whites,,,,Favorable,,,,,,,Major changes,,Yes,,,,,,,Health care,,Yes,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,Latino Democrat,Democratic men,Democrat,Democratic Men,Non-White Democrats,Liberal,,Liberal Democrats,Liberal Democrats,Other,Hispanic/Latino,,,National questionnaire,Hispanic/Latino,Somewhat important,Important,Non-white,Hispanic/Latino,Whites are favored over minorities,West,Catholic,Catholic,,Justified,Male,Latino men,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,2.15388107299805,Non-white,,,,,,


In [1937]:
nat_df['HOU'].value_counts()

3    11535
4     6934
0      319
2      309
1      124
Name: HOU, dtype: int64

In [ ]:
# The Democratic candidate = 3 
# The republican candidate = 4 
# No answer = 0 
# 

### Question: In today's election for U.S. Senate, did you just vote for:
1. The Democratic candidate
2. The Republican candidate
3. Other: Who?
4. Did not vote

In [1651]:
#same idea is the previous transformation 
nat_df = dummy_maker("SEN", nat_df)

### Question: In today's election for governor, did you just vote for:
1. The Democratic candidate
2. The Republican candidate
3. Other: Who?
4. Did not vote

In [1652]:
nat_df = dummy_maker("GOV", nat_df)

### Question: What should the Supreme Court do about Roe vs. Wade, the 1973 decision that made abortion legal?: 
1. Keep as it is
2. Overturn it

In [1653]:
nat_df['ABORTION18'].value_counts()
#verdict: dummy 

                 14485
Keep it as is     3250
Overturn it       1142
Omit               344
Name: ABORTION18, dtype: int64

In [1654]:
nat_df = dummy_maker("ABORTION18", nat_df)

### In which age group are you?: 
1. 18-24
2. 25-29
3. 30-39
4. 40-44
5. 45-49
6. 50-59
7. 60-64
8. 65+ 



In [1655]:
nat_df['AGE'].value_counts()
#verdict dummy 

45-65    7334
30-44    4485
65+      4371
18-29    2908
          123
Name: AGE, dtype: int64

In [1656]:
nat_df = dummy_maker("AGE", nat_df)

### Question:  How often do you attend religious services?: 
1. Once a week or more 
2. A few times a month 
3. A few times a year
4. Never


In [1657]:
nat_df['ATTEND16'].value_counts()

                       14676
Once a week or more     1460
A few times a year      1302
Never                   1163
A few times a month      620
Name: ATTEND16, dtype: int64

In [1658]:
nat_df['ATTEND16'].value_counts()

                       14676
Once a week or more     1460
A few times a year      1302
Never                   1163
A few times a month      620
Name: ATTEND16, dtype: int64

In [1659]:
label_converter(nat_df, 'ATTEND16')

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BOYSGIRLS,BREAK18,BREAK18A,BREAK18B,BRNAGAIN,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+
0,25480,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,0,,,Complete,,,,,,,23,,,,,An advanced degree after a bachelor's degree (...,College graduate,More than high school graduate,White college degree,White women college graduates,,,,Favorable,,,,,,,Minor changes,,Yes,,,,,,,Gun policy,,No,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,White Democrat,Democratic women,Democrat,Democratic Women,White Democrat,Moderate,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,White,White,,,National questionnaire,White,Very important,Important,White,White,Whites are favored over minorities,West,Catholic,Catholic,Catholic,Justified,Female,White women,,,,"Cities 50,000-500,000","Cities over 50,000",English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,1.15386927127838,White Catholics,,,,,,,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0
1,25481,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,Latino 30-44,Latino 30-44,0,,,Complete,,,,,,,40,,,,,Attended college but received no degree,No college degree,More than high school graduate,Non white no college degree,Non-whites,,,,Favorable,,,,,,,Major changes,,Yes,,,,,,,Health care,,Yes,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,Latino Democrat,Democratic men,Democrat,Democratic Men,Non-White Democrats,Liberal,,Liberal Democrats,Liberal Democrats,Other,Hispanic/Latino,,,National questionnaire,Hispanic/Latino,Somewhat important,Important,Non-white,Hispanic/Latino,Whites are favored over minorities,West,Catholic,Catholic,,Justified,Male,Latino men,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,2.15388107299805,Non-white,,,,,,,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0
2,25482,30-59,50-59,45+,50+,45-59,50-64,Black 45-59,Black 45-59,Black 45-64,0,,,Complete,,,,,,,21,,,,,Attended college but received no degree,No college degree,More than high school graduate,Non white no college degree,Non-whites,,,,Omit,,,,,,,Minor changes,,No,,,,,,,Health care,,No,,,Approve,,,45+,18-64,,Democrat,Black Democrat,Black Democrat,Democratic women,Democrat,Democratic Women,Non-White Democrats,Conservative,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,Black,Black,,,National questionnaire,Black,Very important,Important,Non-white,Black,Whites are favored over minorities,West,None,None,,Justified,Female,Black women,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,About right,,Version 2,,,,3.0299916267395,Non-white,,,,,,,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0

In [1660]:
#no_value = 0 
#Once a week or more = 4 
#A few times a year = 3
#Never = 2
#A few times a month = 1

###  A combination of religion and church attendence, will leave alone for now. 

In [1661]:
nat_df['ATTREL'].value_counts()

                                   15087
All others                          2317
Catholic/Do not attend weekly        627
Protestant/Do not attend weekly      474
Protestant/Attend weekly             387
Catholic/Attend weekly               329
Name: ATTREL, dtype: int64

### Do you have any children under 18 living in your home?: 
1. Yes, only boys 
2. Yes, only girls 
3. Yes, both boys and girls 
4. No children in home

In [1662]:
nat_df['BOYSGIRLS'].value_counts()
#verdict dummy 

                            14650
No children in home          3045
Yes, both boys and girls      585
Yes, only boys                485
Yes, only girls               456
Name: BOYSGIRLS, dtype: int64

In [1663]:
nat_df = dummy_maker("BOYSGIRLS", nat_df)

### Question:  In your vote for U.S. House, how would you rate the importance of _______ :
The issue with this question is that the answer on the survey is left blank. Therefore the rank of issue importance is blank as well, making this category useless. 

In [1664]:
nat_df['BREAK18'].value_counts()

                             15173
An important factor           2068
The most important factor      970
Not a factor at all            359
A minor factor                 337
Omit                           314
Name: BREAK18, dtype: int64

In [1665]:
nat_df.loc[nat_df['BREAK18'] == "An important factor"].head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,BRNAGAIN,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls"
3400,29906,30-59,30-39,18-44,18-49,30-44,30-39,Black 30-44,Black 30-44,Black 30-44,0,,,Complete,An important factor,Important,A factor,,Call 1,7,,,,The Democratic Party,Associate's degree (AA or AS),No college degree,More than high school graduate,Non white no college degree,Non-whites,No,,,,,,,No,,,,,,,,,,,,,Oppose,No,,,,,,18-44,18-64,,Democrat,Black Democrat,Black Democrat,Democratic men,Democrat,Democratic Men,Non-White Democrats,Moderate,63,Moderate/Conservative Democrats,Mod/Conserv. Democrats,Black,Black,,,National questionnaire,Black,,,Non-white,Black,,Midwest,,,,,Male,Black men,,,,"Cities over 500,000","Cities over 50,000",,Indiana,Other,,Election day exit poll,,,,,,,Strongly or somewhat disapprove,Strongly disapprove,The U.S. less safe,,,Version 1,No,Hillary Clinton,Should be able to vote will be prevented from ...,.356020212173462,,,,,,,,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
3401,29909,30-59,50-59,45+,50+,45-59,50-64,Non-Black 45-59,White 45-59,White 45-64,0,,,Complete,An important factor,Important,A factor,,Call 1,5,,,,The Republican Party,An advanced degree after a bachelor's degree (...,College graduate,More than high school graduate,White college degree,White men college graduates,No,,,,,,,No,,,,,,,,,,,,,Support,No,,,,,,45+,18-64,,Independent/Something else,Non-black non-Democrat,White Independent,Independent men,Something else,Independent Men,White Independent,Conservative,31,Moderate/Liberal Republicans,Conservative Independents,White,White,,,National questionnaire,White,,,White,White,,South,,,,,Male,White men,,,,Suburbs,Suburbs,,Virginia,White suburban men,,Election day exit poll,,,,,,,Strongly or somewhat approve,Strongly approve,The U.S. more safe,,,Version 1,No,Donald Trump,Shouldn't be able to vote will cast ballots,.525438666343689,,,,,,,,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0


### Question: Would you describe yourself as a born-again or evangelical Christian?: 
1. Yes 
2. No

In [1666]:
nat_df['BRNAGAIN'].value_counts()
#verdict: dummy 

       14598
No      2928
Yes     1695
Name: BRNAGAIN, dtype: int64

In [1667]:
nat_df = dummy_maker("BRNAGAIN", nat_df)

### CALL 
This variable seems to be how many times it took for the surveyer to reach the participant. Excludable 

In [1668]:
nat_df['CALL'].value_counts()

Call 3    5461
Call 1    5289
Call 2    5071
          3400
Name: CALL, dtype: int64

In [1669]:
nat_df.loc[nat_df['CALL'] == "Call 3"].head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes
10295,47451,30-59,50-59,45+,50+,45-59,50-64,Black 45-59,Black 45-59,Black 45-64,0,,,Complete,,,,Call 3,3,,,,,Attended college but received no degree,No college degree,More than high school graduate,Non white no college degree,Non-whites,,Favorable,Only Democratic Party,,Unfavorable,About the same,To express opposition to Donald Trump,,,,,,,"Under $100,000","$30,000 or more","$50,000 or more","$50,000 to $99,999","$50,000-$99,999","Over $50,000 and non white",,,No,No,,,Poor,Not so good or poor,45+,18-64,,Democrat,Black Democrat,Black Democrat,Democratic women,Democrat,Democratic Women,Non-White Democrats,Liberal,112,Liberal Democrats,Liberal Democrats,Black,Black,,,National questionnaire,Black,,,Non-white,Black,,Midwest,,,,,Female,Black women,,,,"Cities over 500,000","Cities over 50,000",,Illinois,Other,Hurt,Election day exit poll,,,,,Seriously off on the wrong track,Hurt,,,,,Becoming more divided,Version 4,,,,0.594259,,No,,,,,,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0
10479,47892,60+,65 or over,45+,50+,60+,65+,Non-Black 60+,Latino 60+,Latino 65+,0,,,Complete,,,,Call 3,3,,,,,Never attended college,No college degree,High school graduate or less,Non white no college degree,Non-whites,,,,Favorable,,,,,,,Major changes,,Yes,,,,,,,The economy,,Yes,,,Approve,,,45+,65+,,Democrat,Non-black Democrat,Latino Democrat,Democratic women,Democrat,Democratic Women,Non-White Democrats,Conservative,112,Moderate/Conservative Democrats,Mod/Conserv. Democrats,Other,Hispanic/Latino,,,National questionnaire,Hispanic/Latino,Very important,Important,Non-white,Hispanic/Latino,Whites are favored over minorities,Midwest,Catholic,Catholic,,Justified,Female,Latino women,,,,"Cities over 500,000","Cities over 50,000",,Illinois,Other,,Election day exit poll,In the last week,In the last month,In the last week,,,,,,,Too tough,,Version 2,,,,1.23354,Non-white,,,,,,,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0


### Candidate Number?

In [1670]:
nat_df['CDNUM'].value_counts()

1     3598
2     2471
3     2215
4     1606
5     1439
8      865
6      839
9      627
12     531
7      514
14     441
26     370
13     362
11     351
15     325
20     221
16     211
10     205
17     205
22     189
25     176
19     129
18     119
30     111
21     100
27      88
32      84
35      80
48      73
52      72
38      72
46      66
39      58
44      52
36      36
24      35
23      35
41      33
33      32
47      24
28      21
42      17
34      16
53      15
29      14
50      13
49      13
45      12
31      12
51      10
43       8
37       5
40       5
Name: CDNUM, dtype: int64

In [1671]:
nat_df.loc[nat_df['CDNUM'] == 1].tail(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes
19205,70280,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,0,,,Complete,,,,Call 3,1,,,,,Bachelor's degree (BA or BS),College graduate,More than high school graduate,White college degree,White men college graduates,,,,Unfavorable,,,,,,,Minor changes,,No,,,,,,,The economy,,No,,,Disapprove,,,18-44,18-64,,Republican,Non-black non-Democrat,All Republicans,Republican men,Republican,Republican Men,White Republican,Conservative,1,Independent/Something else,Conservative Republicans,White,,White,,National questionnaire,White,Not too important,Not important,White,,Minorities are favored over whites,West,None,None,None,Politically motivated,Male,White men,,,,Rural,Small Cities/Rural,,Alaska,Other,,Election day exit poll,In the last month,In the last month,Before that,,,,,,,Not tough enough,,Version 2,,,,0.323932,White none,,,,,,,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
19207,70282,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,0,,,Complete,,,,Call 3,1,,,,,Never attended college,No college degree,High school graduate or less,White no college degree,White men non college graduates,,Omit,Omit,,Omit,About the same,Donald Trump was not a factor,,,,,,,"$100,000 or more","$30,000 or more","$50,000 or more","$100,000","$100,000-$199,999","Over $50,000 and white",,,No,No,,,Good,Excellent or good,18-44,18-64,,Independent/Something else,Non-black non-Democrat,White Independent,Independent men,Independent,Independent Men,White Independent,Liberal,1,Moderate/Liberal Republicans,Liberal Independents,White,,White,,National questionnaire,White,,,White,,,West,,,,,Male,White men,,,,Rural,Small Cities/Rural,,Alaska,Other,Hurt,Election day exit poll,,,,,Generally going in the right direction,Hurt,,,,,Becoming more divided,Version 4,,,,0.216002,,No,,,,,,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0


### Refer to child question 
These are broken down categories for the child question, will label

In [1672]:
nat_df['CHILD18'].value_counts()

       14650
No      3045
Yes     1526
Name: CHILD18, dtype: int64

In [1673]:
nat_df['CHILDBOY'].value_counts()

       14650
No      3501
Yes     1070
Name: CHILDBOY, dtype: int64

In [1674]:
nat_df['CHILDGIRL'].value_counts()

       14650
No      3530
Yes     1041
Name: CHILDGIRL, dtype: int64

In [1675]:
label_converter(nat_df, "CHILD18")
label_converter(nat_df, 'CHILDBOY')
label_converter(nat_df, 'CHILDGIRL')

/Users/nathan/anaconda3/lib/python3.7/site-packages/IPython/core/displayhook.py:271: UserWarning: Output cache limit (currently 1000 entries) hit.
Flushing oldest 200 entries.
  'Flushing oldest {cull_count} entries.'.format(sz=sz, cull_count=cull_count))


,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes
0,25480,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,0,,,Complete,,,,,23,0,0,0,,An advanced degree after a bachelor's degree (...,College graduate,More than high school graduate,White college degree,White women college graduates,,,,Favorable,,,,,,,Minor changes,,Yes,,,,,,,Gun policy,,No,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,White Democrat,Democratic women,Democrat,Democratic Women,White Democrat,Moderate,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,White,White,,,National questionnaire,White,Very important,Important,White,White,Whites are favored over minorities,West,Catholic,Catholic,Catholic,Justified,Female,White women,,,,"Cities 50,000-500,000","Cities over 50,000",English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,1.15386927127838,White Catholics,,,,,,,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,25481,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,Latino 30-44,Latino 30-44,0,,,Complete,,,,,40,0,0,0,,Attended college but received no degree,No college degree,More than high school graduate,Non white no college degree,Non-whites,,,,Favorable,,,,,,,Major changes,,Yes,,,,,,,Health care,,Yes,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,Latino Democrat,Democratic men,Democrat,Democratic Men,Non-White Democrats,Liberal,,Liberal Democrats,Liberal Democrats,Other,Hispanic/Latino,,,National questionnaire,Hispanic/Latino,Somewhat important,Important,Non-white,Hispanic/Latino,Whites are favored over minorities,West,Catholic,Catholic,,Justified,Male,Latino men,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,2.15388107299805,Non-white,,,,,,,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,25482,30-59,50-59,45+,50+,45-59,50-64,Black 45-59,Black 45-59,Black 45-64,0,,,Complete,,,,,21,0,0,0,,Attended college but received no degree,No college degree,More than high school graduate,Non white no college degree,Non-whites,,,,Omit,,,,,,,Minor changes,,No,,,,,,,Health care,,No,,,Approve,,,45+,18-64,,Democrat,Black Democrat,Black Democrat,Democratic women,Democrat,Democratic Women,Non-White Democrats,Conservative,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,Black,Black,,,National questionnaire,Black,Very important,Important,Non-white,Black,Whites are favored over minorities,West,None,None,,Justified,Female,Black women,,,,Suburbs,Suburbs,Englis

In [1676]:
nat_df['CHILDGIRL'].value_counts()

0    14650
1     3530
2     1041
Name: CHILDGIRL, dtype: int64

In [1677]:
#for CHILDGIRL, CHILDBOY, and CHILD 18
# 1 = no 
# 2 = yes
# 0 = no answer 

### Question: Would you rather see the U.S. House of Representatives controlled by:
1. The Democratic Party 
2. The Republican Party
    - V 1

In [1678]:
nat_df['CONTROLHOUSE'].value_counts()
#verdict: label 

                        14485
The Democratic Party     2743
The Republican Party     1757
Omit                      236
Name: CONTROLHOUSE, dtype: int64

In [1679]:
nat_df.loc[nat_df['CONTROLHOUSE'] == "Omit"].head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes
216,25696,18-29,18-24,18-44,18-49,18-29,18-24,Non-Black 18-29,White 18-29,White 18-29,0,,,Complete,,,,,3,0,0,0,Omit,Attended college but received no degree,No college degree,More than high school graduate,White no college degree,White women non college graduates,Omit,,,,,,,Yes,,,,,,,,,,,,,Oppose,No,,,,,,18-44,18-64,,Independent/Something else,Non-black non-Democrat,White Independent,Independent women,Something else,Independent Women,White Independent,Moderate,,Moderate/Liberal Republicans,Moderate Independents,White,White,,,National questionnaire,White,,,White,White,,West,,,,,Female,White women,,,,"Cities 50,000-500,000","Cities over 50,000",English,California,Other,,Telephone survey,,,,,,,Strongly or somewhat disapprove,Strongly disapprove,The U.S. less safe,,,Version 1,No,Didn't vote,Should be able to vote will be prevented from ...,3.45989918708801,,,,,,,,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0
228,25708,18-29,25-29,18-44,18-49,18-29,25-29,Non-Black 18-29,All others,All others,0,,,Complete,,,,,39,0,0,0,Omit,Bachelor's degree (BA or BS),College graduate,More than high school graduate,Non white college degree,Non-whites,No,,,,,,,No,,,,,,,,,,,,,Oppose,No,,,,,,18-44,18-64,,Independent/Something else,Non-black non-Democrat,All Other,Independent men,Independent,Independent Men,All others,Moderate,,Moderate/Liberal Republicans,Moderate Independents,Other,Asian,,,National questionnaire,Asian,,,Non-white,Asian,,West,,,,,Male,All other races,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,,,,Strongly or somewhat disapprove,Somewhat disapprove,The U.S. less safe,,,Version 1,No,Other,Should be able to vote will be prevented from ...,9.7637357711792,,,,,,,,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0


In [1680]:
label_converter(nat_df, "CONTROLHOUSE").head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes
0,25480,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,0,,,Complete,,,,,23,0,0,0,0,An advanced degree after a bachelor's degree (...,College graduate,More than high school graduate,White college degree,White women college graduates,,,,Favorable,,,,,,,Minor changes,,Yes,,,,,,,Gun policy,,No,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,White Democrat,Democratic women,Democrat,Democratic Women,White Democrat,Moderate,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,White,White,,,National questionnaire,White,Very important,Important,White,White,Whites are favored over minorities,West,Catholic,Catholic,Catholic,Justified,Female,White women,,,,"Cities 50,000-500,000","Cities over 50,000",English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,1.15386927127838,White Catholics,,,,,,,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,25481,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,Latino 30-44,Latino 30-44,0,,,Complete,,,,,40,0,0,0,0,Attended college but received no degree,No college degree,More than high school graduate,Non white no college degree,Non-whites,,,,Favorable,,,,,,,Major changes,,Yes,,,,,,,Health care,,Yes,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,Latino Democrat,Democratic men,Democrat,Democratic Men,Non-White Democrats,Liberal,,Liberal Democrats,Liberal Democrats,Other,Hispanic/Latino,,,National questionnaire,Hispanic/Latino,Somewhat important,Important,Non-white,Hispanic/Latino,Whites are favored over minorities,West,Catholic,Catholic,,Justified,Male,Latino men,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,2.15388107299805,Non-white,,,,,,,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0


### Question: Which best describes your education? You have:
1. Bachelor's degree (BA or BS)                                                   
2. Attended college but received no degree                                        
3. An advanced degree after a bachelor's degree (such as JD, MA, MBA, MD, PhD)    
4. Never attended college                                                         
5. Associate's degree (AA or AS)     
#### Section includes the variables EDUC18, EDUCCOLL,  and EDUCHS

In [1681]:
nat_df["EDUC18"].value_counts()
#label

Bachelor's degree (BA or BS)                                                   5135
Attended college but received no degree                                        4539
An advanced degree after a bachelor's degree (such as JD, MA, MBA, MD, PhD)    3829
Never attended college                                                         2934
Associate's degree (AA or AS)                                                  2485
                                                                                299
Name: EDUC18, dtype: int64

In [1682]:
label_converter(nat_df, "EDUC18")
label_converter(nat_df, "EDUCCOLL")
label_converter(nat_df, "EDUCHS")

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes
0,25480,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,0,,,Complete,,,,,23,0,0,0,0,1,1,2,White college degree,White women college graduates,,,,Favorable,,,,,,,Minor changes,,Yes,,,,,,,Gun policy,,No,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,White Democrat,Democratic women,Democrat,Democratic Women,White Democrat,Moderate,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,White,White,,,National questionnaire,White,Very important,Important,White,White,Whites are favored over minorities,West,Catholic,Catholic,Catholic,Justified,Female,White women,,,,"Cities 50,000-500,000","Cities over 50,000",English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,1.15386927127838,White Catholics,,,,,,,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,25481,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,Latino 30-44,Latino 30-44,0,,,Complete,,,,,40,0,0,0,0,3,2,2,Non white no college degree,Non-whites,,,,Favorable,,,,,,,Major changes,,Yes,,,,,,,Health care,,Yes,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,Latino Democrat,Democratic men,Democrat,Democratic Men,Non-White Democrats,Liberal,,Liberal Democrats,Liberal Democrats,Other,Hispanic/Latino,,,National questionnaire,Hispanic/Latino,Somewhat important,Important,Non-white,Hispanic/Latino,Whites are favored over minorities,West,Catholic,Catholic,,Justified,Male,Latino men,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,2.15388107299805,Non-white,,,,,,,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,25482,30-59,50-59,45+,50+,45-59,50-64,Black 45-59,Black 45-59,Black 45-64,0,,,Complete,,,,,21,0,0,0,0,3,2,2,Non white no college degree,Non-whites,,,,Omit,,,,,,,Minor changes,,No,,,,,,,Health care,,No,,,Approve,,,45+,18-64,,Democrat,Black Democrat,Black Democrat,Democratic women,Democrat,Democratic Women,Non-White Democrats,Conservative,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,Black,Black,,,National questionnaire,Black,Very important,Important,Non-white,Black,Whites are favored over minorities,West,None,None,,Justified,Female,Black women,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,About right,,Version 2,,,,3.0299916267395,Non-white,,,,,,,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,25483,60+,65 or over,45+,50+,60+,65+,Non-Black 60+,White 60+,White 65+,0,,,Complete,,

In [1683]:
nat_df['EDUCCOLL'].value_counts()

2    9958
1    8964
0     299
Name: EDUCCOLL, dtype: int64

In [1684]:
nat_df['EDUCCOLL'].value_counts()

2    9958
1    8964
0     299
Name: EDUCCOLL, dtype: int64

In [1685]:
#no college degree = 2 
#college graduate = 1
#no answer = 0 

In [1686]:
nat_df['EDUCHS'].value_counts()

2    15988
1     2934
0      299
Name: EDUCHS, dtype: int64

In [1687]:
nat_df['EDUCHS'].value_counts()

2    15988
1     2934
0      299
Name: EDUCHS, dtype: int64

In [1688]:
#more than high school graduate = 2 
#high school graduate or less = 1
#no answer = 0 

### Question: Do you think the government has done enough to protect this election from foreign interference?: 
1. Yes
2. No

In [1689]:
nat_df['ELECTSECFOREIGN18'].value_counts()
#label

        14485
No       2649
Yes      1648
Omit      439
Name: ELECTSECFOREIGN18, dtype: int64

In [1690]:
label_converter(nat_df, 'ELECTSECFOREIGN18')

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes
0,25480,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,0,,,Complete,,,,,23,0,0,0,0,1,1,2,White college degree,White women college graduates,0,,,Favorable,,,,,,,Minor changes,,Yes,,,,,,,Gun policy,,No,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,White Democrat,Democratic women,Democrat,Democratic Women,White Democrat,Moderate,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,White,White,,,National questionnaire,White,Very important,Important,White,White,Whites are favored over minorities,West,Catholic,Catholic,Catholic,Justified,Female,White women,,,,"Cities 50,000-500,000","Cities over 50,000",English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,1.15386927127838,White Catholics,,,,,,,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,25481,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,Latino 30-44,Latino 30-44,0,,,Complete,,,,,40,0,0,0,0,3,2,2,Non white no college degree,Non-whites,0,,,Favorable,,,,,,,Major changes,,Yes,,,,,,,Health care,,Yes,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,Latino Democrat,Democratic men,Democrat,Democratic Men,Non-White Democrats,Liberal,,Liberal Democrats,Liberal Democrats,Other,Hispanic/Latino,,,National questionnaire,Hispanic/Latino,Somewhat important,Important,Non-white,Hispanic/Latino,Whites are favored over minorities,West,Catholic,Catholic,,Justified,Male,Latino men,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,2.15388107299805,Non-white,,,,,,,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,25482,30-59,50-59,45+,50+,45-59,50-64,Black 45-59,Black 45-59,Black 45-64,0,,,Complete,,,,,21,0,0,0,0,3,2,2,Non white no college degree,Non-whites,0,,,Omit,,,,,,,Minor changes,,No,,,,,,,Health care,,No,,,Approve,,,45+,18-64,,Democrat,Black Democrat,Black Democrat,Democratic women,Democrat,Democratic Women,Non-White Democrats,Conservative,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,Black,Black,,,National questionnaire,Black,Very important,Important,Non-white,Black,Whites are favored over minorities,West,None,None,,Justified,Female,Black women,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,About right,,Version 2,,,,3.0299916267395,Non-white,,,,,,,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,25483,60+,65 or over,45+,50+,60+,65+,Non-Black 60+,White 60+,White 65+,0,,,Complet

In [1691]:
nat_df['ELECTSECFOREIGN18'].value_counts()

0    14485
1     2649
3     1648
2      439
Name: ELECTSECFOREIGN18, dtype: int64

In [1692]:
# No answer = 0
# no = 1
# yes = 3
# ommit = 2

### Question:  Is your opinion of the Democratic Party:
1. Favorable
2. Unfavorable 
 - V = 4 

In [1693]:
nat_df['FAVDEM2'].value_counts()
#label

               14635
Favorable       2547
Unfavorable     1868
Omit             171
Name: FAVDEM2, dtype: int64

In [1694]:
label_converter(nat_df, "FAVDEM2")

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes
0,25480,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,0,,,Complete,,,,,23,0,0,0,0,1,1,2,White college degree,White women college graduates,0,0,,Favorable,,,,,,,Minor changes,,Yes,,,,,,,Gun policy,,No,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,White Democrat,Democratic women,Democrat,Democratic Women,White Democrat,Moderate,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,White,White,,,National questionnaire,White,Very important,Important,White,White,Whites are favored over minorities,West,Catholic,Catholic,Catholic,Justified,Female,White women,,,,"Cities 50,000-500,000","Cities over 50,000",English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,1.15386927127838,White Catholics,,,,,,,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,25481,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,Latino 30-44,Latino 30-44,0,,,Complete,,,,,40,0,0,0,0,3,2,2,Non white no college degree,Non-whites,0,0,,Favorable,,,,,,,Major changes,,Yes,,,,,,,Health care,,Yes,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,Latino Democrat,Democratic men,Democrat,Democratic Men,Non-White Democrats,Liberal,,Liberal Democrats,Liberal Democrats,Other,Hispanic/Latino,,,National questionnaire,Hispanic/Latino,Somewhat important,Important,Non-white,Hispanic/Latino,Whites are favored over minorities,West,Catholic,Catholic,,Justified,Male,Latino men,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,2.15388107299805,Non-white,,,,,,,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,25482,30-59,50-59,45+,50+,45-59,50-64,Black 45-59,Black 45-59,Black 45-64,0,,,Complete,,,,,21,0,0,0,0,3,2,2,Non white no college degree,Non-whites,0,0,,Omit,,,,,,,Minor changes,,No,,,,,,,Health care,,No,,,Approve,,,45+,18-64,,Democrat,Black Democrat,Black Democrat,Democratic women,Democrat,Democratic Women,Non-White Democrats,Conservative,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,Black,Black,,,National questionnaire,Black,Very important,Important,Non-white,Black,Whites are favored over minorities,West,None,None,,Justified,Female,Black women,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,About right,,Version 2,,,,3.0299916267395,Non-white,,,,,,,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,25483,60+,65 or over,45+,50+,60+,65+,Non-Black 60+,White 60+,White 65+,0,,,Comp

In [1695]:
nat_df['FAVDEM2'].value_counts()


0    14635
1     2547
3     1868
2      171
Name: FAVDEM2, dtype: int64

In [1696]:
#no answer = 0 
# favorable = 1
# unfavorable = 3
# omit = 1

### Question:  Is your opinion of the Republican Party:
1. Favorable
2. Unfavorable 
 - V = 4 

In [1697]:
nat_df['FAVREP2'].value_counts()
#label 

               14635
Unfavorable     2718
Favorable       1686
Omit             182
Name: FAVREP2, dtype: int64

In [1698]:
label_converter(nat_df, "FAVREP2")

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes
0,25480,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,0,,,Complete,,,,,23,0,0,0,0,1,1,2,White college degree,White women college graduates,0,0,,Favorable,0,,,,,,Minor changes,,Yes,,,,,,,Gun policy,,No,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,White Democrat,Democratic women,Democrat,Democratic Women,White Democrat,Moderate,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,White,White,,,National questionnaire,White,Very important,Important,White,White,Whites are favored over minorities,West,Catholic,Catholic,Catholic,Justified,Female,White women,,,,"Cities 50,000-500,000","Cities over 50,000",English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,1.15386927127838,White Catholics,,,,,,,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,25481,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,Latino 30-44,Latino 30-44,0,,,Complete,,,,,40,0,0,0,0,3,2,2,Non white no college degree,Non-whites,0,0,,Favorable,0,,,,,,Major changes,,Yes,,,,,,,Health care,,Yes,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,Latino Democrat,Democratic men,Democrat,Democratic Men,Non-White Democrats,Liberal,,Liberal Democrats,Liberal Democrats,Other,Hispanic/Latino,,,National questionnaire,Hispanic/Latino,Somewhat important,Important,Non-white,Hispanic/Latino,Whites are favored over minorities,West,Catholic,Catholic,,Justified,Male,Latino men,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,2.15388107299805,Non-white,,,,,,,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,25482,30-59,50-59,45+,50+,45-59,50-64,Black 45-59,Black 45-59,Black 45-64,0,,,Complete,,,,,21,0,0,0,0,3,2,2,Non white no college degree,Non-whites,0,0,,Omit,0,,,,,,Minor changes,,No,,,,,,,Health care,,No,,,Approve,,,45+,18-64,,Democrat,Black Democrat,Black Democrat,Democratic women,Democrat,Democratic Women,Non-White Democrats,Conservative,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,Black,Black,,,National questionnaire,Black,Very important,Important,Non-white,Black,Whites are favored over minorities,West,None,None,,Justified,Female,Black women,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,About right,,Version 2,,,,3.0299916267395,Non-white,,,,,,,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,25483,60+,65 or over,45+,50+,60+,65+,Non-Black 60+,White 60+,White 65+,0,,,C

In [1699]:
#no answer = 0 
# favorable = 1
# unfavorable = 3
# omit = 1

### A feature interaction in combination with the question above 

In [1700]:
nat_df['FAVPARTY'].value_counts()
#label 

                         14635
Only Democratic Party     2233
Only Republican Party     1405
Neither party              437
Omit                       262
Both parties               249
Name: FAVPARTY, dtype: int64

In [1701]:
nat_df.loc[nat_df['FAVPARTY'] == "Omit"].head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes
110,25590,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,0,,,Complete,,,,,29,0,0,0,0,4,1,2,White college degree,White men college graduates,0,3,Omit,,2,About the same,Donald Trump was not a factor,,,,,,,"$100,000 or more","$30,000 or more","$50,000 or more","$100,000",$200.000 or more,"Over $50,000 and white",,,No,No,,,Not so good,Not so good or poor,18-44,18-64,,Republican,Non-black non-Democrat,All Republicans,Republican men,Republican,Republican Men,White Republican,Conservative,,Independent/Something else,Conservative Republicans,White,White,,,National questionnaire,White,,,White,White,,West,,,,,Male,White men,,,,Suburbs,Suburbs,English,California,White suburban men,No impact,Telephone survey,,,,,Seriously off on the wrong track,No impact,,,,,Staying about the same,Version 4,,,,3.14571619033813,,No,,,,,,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
136,25616,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,0,,,Complete,,,,,24,0,0,0,0,1,1,2,White college degree,White men college graduates,0,2,Omit,,3,About the same,Donald Trump was not a factor,,,,,,,"$100,000 or more","$30,000 or more","$50,000 or more","$100,000","$100,000-$199,999","Over $50,000 and white",,,No,No,,,Good,Excellent or good,18-44,18-64,,Independent/Something else,Non-black non-Democrat,White Independent,Independent men,Something else,Independent Men,White Independent,Liberal,,Moderate/Liberal Republicans,Liberal Independents,White,White,,,National questionnaire,White,,,White,White,,West,,,,,Male,White men,,,,"Cities 10,000-50,000",Small Cities/Rural,English,California,Other,No impact,Telephone survey,,,,,Seriously off on the wrong track,Hurt,,,,,Becoming more divided,Version 4,,,,1.81864798069,,No,,,,,,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [1702]:
label_converter(nat_df, "FAVPARTY").head(3)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes
0,25480,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,0,,,Complete,,,,,23,0,0,0,0,1,1,2,White college degree,White women college graduates,0,0,0,Favorable,0,,,,,,Minor changes,,Yes,,,,,,,Gun policy,,No,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,White Democrat,Democratic women,Democrat,Democratic Women,White Democrat,Moderate,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,White,White,,,National questionnaire,White,Very important,Important,White,White,Whites are favored over minorities,West,Catholic,Catholic,Catholic,Justified,Female,White women,,,,"Cities 50,000-500,000","Cities over 50,000",English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,1.15386927127838,White Catholics,,,,,,,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,25481,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,Latino 30-44,Latino 30-44,0,,,Complete,,,,,40,0,0,0,0,3,2,2,Non white no college degree,Non-whites,0,0,0,Favorable,0,,,,,,Major changes,,Yes,,,,,,,Health care,,Yes,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,Latino Democrat,Democratic men,Democrat,Democratic Men,Non-White Democrats,Liberal,,Liberal Democrats,Liberal Democrats,Other,Hispanic/Latino,,,National questionnaire,Hispanic/Latino,Somewhat important,Important,Non-white,Hispanic/Latino,Whites are favored over minorities,West,Catholic,Catholic,,Justified,Male,Latino men,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,2.15388107299805,Non-white,,,,,,,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,25482,30-59,50-59,45+,50+,45-59,50-64,Black 45-59,Black 45-59,Black 45-64,0,,,Complete,,,,,21,0,0,0,0,3,2,2,Non white no college degree,Non-whites,0,0,0,Omit,0,,,,,,Minor changes,,No,,,,,,,Health care,,No,,,Approve,,,45+,18-64,,Democrat,Black Democrat,Black Democrat,Democratic women,Democrat,Democratic Women,Non-White Democrats,Conservative,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,Black,Black,,,National questionnaire,Black,Very important,Important,Non-white,Black,Whites are favored over minorities,West,None,None,,Justified,Female,Black women,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,About right,,Version 2,,,,3.0299916267395,Non-white,,,,,,,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [1703]:
nat_df['FAVPARTY'].value_counts()

0    14635
4     2233
5     1405
2      437
3      262
1      249
Name: FAVPARTY, dtype: int64

In [1704]:
#no answer = 0 
#Only Dem = 4
#Only Rep = 1405
#Neither Party = 437 
#Omit = 262
#Both parties = 249 

### Is your opinion of Nancy Pelosi:
1. Favorable
2. Unfavorable 
 - V = 2

In [1705]:
nat_df['FAVPELOSI'].value_counts()
#label 

               14548
Unfavorable     2419
Favorable       1629
Omit             625
Name: FAVPELOSI, dtype: int64

In [1706]:
label_converter(nat_df, "FAVPELOSI")

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes
0,25480,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,0,,,Complete,,,,,23,0,0,0,0,1,1,2,White college degree,White women college graduates,0,0,0,1,0,,,,,,Minor changes,,Yes,,,,,,,Gun policy,,No,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,White Democrat,Democratic women,Democrat,Democratic Women,White Democrat,Moderate,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,White,White,,,National questionnaire,White,Very important,Important,White,White,Whites are favored over minorities,West,Catholic,Catholic,Catholic,Justified,Female,White women,,,,"Cities 50,000-500,000","Cities over 50,000",English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,1.15386927127838,White Catholics,,,,,,,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,25481,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,Latino 30-44,Latino 30-44,0,,,Complete,,,,,40,0,0,0,0,3,2,2,Non white no college degree,Non-whites,0,0,0,1,0,,,,,,Major changes,,Yes,,,,,,,Health care,,Yes,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,Latino Democrat,Democratic men,Democrat,Democratic Men,Non-White Democrats,Liberal,,Liberal Democrats,Liberal Democrats,Other,Hispanic/Latino,,,National questionnaire,Hispanic/Latino,Somewhat important,Important,Non-white,Hispanic/Latino,Whites are favored over minorities,West,Catholic,Catholic,,Justified,Male,Latino men,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,2.15388107299805,Non-white,,,,,,,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,25482,30-59,50-59,45+,50+,45-59,50-64,Black 45-59,Black 45-59,Black 45-64,0,,,Complete,,,,,21,0,0,0,0,3,2,2,Non white no college degree,Non-whites,0,0,0,2,0,,,,,,Minor changes,,No,,,,,,,Health care,,No,,,Approve,,,45+,18-64,,Democrat,Black Democrat,Black Democrat,Democratic women,Democrat,Democratic Women,Non-White Democrats,Conservative,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,Black,Black,,,National questionnaire,Black,Very important,Important,Non-white,Black,Whites are favored over minorities,West,None,None,,Justified,Female,Black women,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,About right,,Version 2,,,,3.0299916267395,Non-white,,,,,,,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,25483,60+,65 or over,45+,50+,60+,65+,Non-Black 60+,White 60+,White 65+,0,,,Complete,,,,,1,0,

In [1707]:
nat_df['FAVPELOSI'].value_counts()

0    14548
3     2419
1     1629
2      625
Name: FAVPELOSI, dtype: int64

In [1708]:
#no answer = 0 
#unfavorable = 3 
#favorable = 1
#omit = 2

### Question: Compared to two years ago, is your family's financial situation:
1. Better 
2. Worse 
3. About the same 
  - V 4 

In [1709]:
nat_df['FINSIT2'].value_counts()
#label

                  14635
About the same     2250
Better             1586
Worse               707
Omit                 43
Name: FINSIT2, dtype: int64

In [1710]:
label_converter(nat_df, "FINSIT2").head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes
0,25480,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,0,,,Complete,,,,,23,0,0,0,0,1,1,2,White college degree,White women college graduates,0,0,0,1,0,0,,,,,Minor changes,,Yes,,,,,,,Gun policy,,No,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,White Democrat,Democratic women,Democrat,Democratic Women,White Democrat,Moderate,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,White,White,,,National questionnaire,White,Very important,Important,White,White,Whites are favored over minorities,West,Catholic,Catholic,Catholic,Justified,Female,White women,,,,"Cities 50,000-500,000","Cities over 50,000",English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,1.15386927127838,White Catholics,,,,,,,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,25481,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,Latino 30-44,Latino 30-44,0,,,Complete,,,,,40,0,0,0,0,3,2,2,Non white no college degree,Non-whites,0,0,0,1,0,0,,,,,Major changes,,Yes,,,,,,,Health care,,Yes,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,Latino Democrat,Democratic men,Democrat,Democratic Men,Non-White Democrats,Liberal,,Liberal Democrats,Liberal Democrats,Other,Hispanic/Latino,,,National questionnaire,Hispanic/Latino,Somewhat important,Important,Non-white,Hispanic/Latino,Whites are favored over minorities,West,Catholic,Catholic,,Justified,Male,Latino men,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,2.15388107299805,Non-white,,,,,,,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [1711]:
nat_df['FINSIT2'].value_counts()

0    14635
1     2250
2     1586
4      707
3       43
Name: FINSIT2, dtype: int64

In [1712]:
# no answer = 0 
# About the same = 1
# better = 2
# worse = 3
# omit = 4


### Was one reason for your vote for U.S. House today:
1. To express support for Donald Trump
2. To express opposition to Donald Trump 
3. Donald Trump was not a factor 
    - V 4

In [1713]:
nat_df['FORTRUMPH'].value_counts()
#label 

                                         14507
To express opposition to Donald Trump     2031
Donald Trump was not a factor             1529
To express support for Donald Trump       1021
Omit                                       133
Name: FORTRUMPH, dtype: int64

In [1714]:
label_converter(nat_df, "FORTRUMPH").head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes
0,25480,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,0,,,Complete,,,,,23,0,0,0,0,1,1,2,White college degree,White women college graduates,0,0,0,1,0,0,0,,,,Minor changes,,Yes,,,,,,,Gun policy,,No,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,White Democrat,Democratic women,Democrat,Democratic Women,White Democrat,Moderate,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,White,White,,,National questionnaire,White,Very important,Important,White,White,Whites are favored over minorities,West,Catholic,Catholic,Catholic,Justified,Female,White women,,,,"Cities 50,000-500,000","Cities over 50,000",English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,1.15386927127838,White Catholics,,,,,,,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,25481,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,Latino 30-44,Latino 30-44,0,,,Complete,,,,,40,0,0,0,0,3,2,2,Non white no college degree,Non-whites,0,0,0,1,0,0,0,,,,Major changes,,Yes,,,,,,,Health care,,Yes,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,Latino Democrat,Democratic men,Democrat,Democratic Men,Non-White Democrats,Liberal,,Liberal Democrats,Liberal Democrats,Other,Hispanic/Latino,,,National questionnaire,Hispanic/Latino,Somewhat important,Important,Non-white,Hispanic/Latino,Whites are favored over minorities,West,Catholic,Catholic,,Justified,Male,Latino men,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,2.15388107299805,Non-white,,,,,,,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [1715]:
nat_df['FORTRUMPH'].value_counts()

0    14507
3     2031
1     1529
4     1021
2      133
Name: FORTRUMPH, dtype: int64

In [1716]:
# no answer = 0 
# To express opposition = 3
# Donald Trump was not a factor = 1
# To express support = 4
# omit = 2 

###  Is this the first time you have ever voted in a midterm election?: 
1. Yes 
2. No 
    - V 1

In [1717]:
nat_df['FTVOTER18'].value_counts()
#label 

       14523
No      3909
Yes      789
Name: FTVOTER18, dtype: int64

In [1718]:
label_converter(nat_df, 'FTVOTER18').head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes
0,25480,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,0,,,Complete,,,,,23,0,0,0,0,1,1,2,White college degree,White women college graduates,0,0,0,1,0,0,0,0,,,Minor changes,,Yes,,,,,,,Gun policy,,No,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,White Democrat,Democratic women,Democrat,Democratic Women,White Democrat,Moderate,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,White,White,,,National questionnaire,White,Very important,Important,White,White,Whites are favored over minorities,West,Catholic,Catholic,Catholic,Justified,Female,White women,,,,"Cities 50,000-500,000","Cities over 50,000",English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,1.15386927127838,White Catholics,,,,,,,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,25481,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,Latino 30-44,Latino 30-44,0,,,Complete,,,,,40,0,0,0,0,3,2,2,Non white no college degree,Non-whites,0,0,0,1,0,0,0,0,,,Major changes,,Yes,,,,,,,Health care,,Yes,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,Latino Democrat,Democratic men,Democrat,Democratic Men,Non-White Democrats,Liberal,,Liberal Democrats,Liberal Democrats,Other,Hispanic/Latino,,,National questionnaire,Hispanic/Latino,Somewhat important,Important,Non-white,Hispanic/Latino,Whites are favored over minorities,West,Catholic,Catholic,,Justified,Male,Latino men,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,2.15388107299805,Non-white,,,,,,,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [1719]:
nat_df['FTVOTER18'].value_counts()

0    14523
1     3909
2      789
Name: FTVOTER18, dtype: int64

In [1720]:
# No answer = 0
# no = 1
# yes = 2 

### Question: Do you support or oppose stricter gun control measures?: 
1. Support 
2. Oppose 
    - V 3

In [1721]:
nat_df['GUN16'].value_counts()
#label

           14594
Support     2921
Oppose      1493
Omit         213
Name: GUN16, dtype: int64

In [1722]:
label_converter(nat_df, 'GUN16').head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes
0,25480,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,0,,,Complete,,,,,23,0,0,0,0,1,1,2,White college degree,White women college graduates,0,0,0,1,0,0,0,0,0,,Minor changes,,Yes,,,,,,,Gun policy,,No,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,White Democrat,Democratic women,Democrat,Democratic Women,White Democrat,Moderate,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,White,White,,,National questionnaire,White,Very important,Important,White,White,Whites are favored over minorities,West,Catholic,Catholic,Catholic,Justified,Female,White women,,,,"Cities 50,000-500,000","Cities over 50,000",English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,1.15386927127838,White Catholics,,,,,,,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,25481,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,Latino 30-44,Latino 30-44,0,,,Complete,,,,,40,0,0,0,0,3,2,2,Non white no college degree,Non-whites,0,0,0,1,0,0,0,0,0,,Major changes,,Yes,,,,,,,Health care,,Yes,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,Latino Democrat,Democratic men,Democrat,Democratic Men,Non-White Democrats,Liberal,,Liberal Democrats,Liberal Democrats,Other,Hispanic/Latino,,,National questionnaire,Hispanic/Latino,Somewhat important,Important,Non-white,Hispanic/Latino,Whites are favored over minorities,West,Catholic,Catholic,,Justified,Male,Latino men,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,2.15388107299805,Non-white,,,,,,,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [1723]:
nat_df['GUN16'].value_counts()

0    14594
3     2921
2     1493
1      213
Name: GUN16, dtype: int64

In [1724]:
# no answer = 0
# support = 3
# oppose = 1493
# omit = 1 

### Does anyone in your household own a gun?: 
1. Yes 
2. No
    - V 3

In [1725]:
nat_df['GUNOWNER12'].value_counts()
#label

       14568
No      2611
Yes     2042
Name: GUNOWNER12, dtype: int64

In [1726]:
label_converter(nat_df, 'GUNOWNER12').head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes
0,25480,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,0,,,Complete,,,,,23,0,0,0,0,1,1,2,White college degree,White women college graduates,0,0,0,1,0,0,0,0,0,0,Minor changes,,Yes,,,,,,,Gun policy,,No,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,White Democrat,Democratic women,Democrat,Democratic Women,White Democrat,Moderate,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,White,White,,,National questionnaire,White,Very important,Important,White,White,Whites are favored over minorities,West,Catholic,Catholic,Catholic,Justified,Female,White women,,,,"Cities 50,000-500,000","Cities over 50,000",English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,1.15386927127838,White Catholics,,,,,,,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,25481,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,Latino 30-44,Latino 30-44,0,,,Complete,,,,,40,0,0,0,0,3,2,2,Non white no college degree,Non-whites,0,0,0,1,0,0,0,0,0,0,Major changes,,Yes,,,,,,,Health care,,Yes,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,Latino Democrat,Democratic men,Democrat,Democratic Men,Non-White Democrats,Liberal,,Liberal Democrats,Liberal Democrats,Other,Hispanic/Latino,,,National questionnaire,Hispanic/Latino,Somewhat important,Important,Non-white,Hispanic/Latino,Whites are favored over minorities,West,Catholic,Catholic,,Justified,Male,Latino men,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,2.15388107299805,Non-white,,,,,,,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [1727]:
nat_df['GUNOWNER12'].value_counts()

0    14568
1     2611
2     2042
Name: GUNOWNER12, dtype: int64

In [1728]:
# no answer = 0 
# no = 1
# yes = 2 

### Does the health care system in the U.S. need: 
1. Major changes 
2. Minor changes 
3. No changes at all 
    - V 2 

In [1729]:
nat_df['HEALTHCARE18'].value_counts()
# label 

                     14548
Major changes         3338
Minor changes         1034
No changes at all      166
Omit                   135
Name: HEALTHCARE18, dtype: int64

In [1730]:
label_converter(nat_df,'HEALTHCARE18' ).head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes
0,25480,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,0,,,Complete,,,,,23,0,0,0,0,1,1,2,White college degree,White women college graduates,0,0,0,1,0,0,0,0,0,0,2,,Yes,,,,,,,Gun policy,,No,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,White Democrat,Democratic women,Democrat,Democratic Women,White Democrat,Moderate,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,White,White,,,National questionnaire,White,Very important,Important,White,White,Whites are favored over minorities,West,Catholic,Catholic,Catholic,Justified,Female,White women,,,,"Cities 50,000-500,000","Cities over 50,000",English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,1.15386927127838,White Catholics,,,,,,,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,25481,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,Latino 30-44,Latino 30-44,0,,,Complete,,,,,40,0,0,0,0,3,2,2,Non white no college degree,Non-whites,0,0,0,1,0,0,0,0,0,0,1,,Yes,,,,,,,Health care,,Yes,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,Latino Democrat,Democratic men,Democrat,Democratic Men,Non-White Democrats,Liberal,,Liberal Democrats,Liberal Democrats,Other,Hispanic/Latino,,,National questionnaire,Hispanic/Latino,Somewhat important,Important,Non-white,Hispanic/Latino,Whites are favored over minorities,West,Catholic,Catholic,,Justified,Male,Latino men,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,2.15388107299805,Non-white,,,,,,,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [1731]:
nat_df['HEALTHCARE18'].value_counts()

0    14548
1     3338
2     1034
3      166
4      135
Name: HEALTHCARE18, dtype: int64

In [1732]:
# no answer = 0 
# Major Changes = 1 
# Minor Changes = 2 
# No changes at all = 3 
# Omit = 4 

### Which party would better protect health care for people with pre-existing conditions?: 
1. Democratic Party
2. The Republican Party 

In [1733]:
nat_df["HEALTHPRE18"].value_counts()

                        14594
The Democratic Party     2946
The Republican Party     1362
Omit                      319
Name: HEALTHPRE18, dtype: int64

In [1734]:
label_converter(nat_df, 'HEALTHPRE18').head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes
0,25480,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,0,,,Complete,,,,,23,0,0,0,0,1,1,2,White college degree,White women college graduates,0,0,0,1,0,0,0,0,0,0,2,0,Yes,,,,,,,Gun policy,,No,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,White Democrat,Democratic women,Democrat,Democratic Women,White Democrat,Moderate,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,White,White,,,National questionnaire,White,Very important,Important,White,White,Whites are favored over minorities,West,Catholic,Catholic,Catholic,Justified,Female,White women,,,,"Cities 50,000-500,000","Cities over 50,000",English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,1.15386927127838,White Catholics,,,,,,,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,25481,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,Latino 30-44,Latino 30-44,0,,,Complete,,,,,40,0,0,0,0,3,2,2,Non white no college degree,Non-whites,0,0,0,1,0,0,0,0,0,0,1,0,Yes,,,,,,,Health care,,Yes,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,Latino Democrat,Democratic men,Democrat,Democratic Men,Non-White Democrats,Liberal,,Liberal Democrats,Liberal Democrats,Other,Hispanic/Latino,,,National questionnaire,Hispanic/Latino,Somewhat important,Important,Non-white,Hispanic/Latino,Whites are favored over minorities,West,Catholic,Catholic,,Justified,Male,Latino men,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,2.15388107299805,Non-white,,,,,,,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [1735]:
nat_df["HEALTHPRE18"].value_counts()

0    14594
2     2946
3     1362
1      319
Name: HEALTHPRE18, dtype: int64

In [1736]:
# No answer = 0 
# The Democratic Party = 2 
# The Republican Party = 3 
# Omit = 1 

### Question: Should Congress impeach Donald Trump and remove him from office:
1. Yes 
2. No 
    -V 2

In [1737]:
nat_df['IMPEACH18'].value_counts()
#label 

        14548
No       2222
Yes      2203
Omit      248
Name: IMPEACH18, dtype: int64

In [1738]:
label_converter(nat_df, 'IMPEACH18').head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes
0,25480,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,0,,,Complete,,,,,23,0,0,0,0,1,1,2,White college degree,White women college graduates,0,0,0,1,0,0,0,0,0,0,2,0,3,,,,,,,Gun policy,,No,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,White Democrat,Democratic women,Democrat,Democratic Women,White Democrat,Moderate,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,White,White,,,National questionnaire,White,Very important,Important,White,White,Whites are favored over minorities,West,Catholic,Catholic,Catholic,Justified,Female,White women,,,,"Cities 50,000-500,000","Cities over 50,000",English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,1.15386927127838,White Catholics,,,,,,,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,25481,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,Latino 30-44,Latino 30-44,0,,,Complete,,,,,40,0,0,0,0,3,2,2,Non white no college degree,Non-whites,0,0,0,1,0,0,0,0,0,0,1,0,3,,,,,,,Health care,,Yes,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,Latino Democrat,Democratic men,Democrat,Democratic Men,Non-White Democrats,Liberal,,Liberal Democrats,Liberal Democrats,Other,Hispanic/Latino,,,National questionnaire,Hispanic/Latino,Somewhat important,Important,Non-white,Hispanic/Latino,Whites are favored over minorities,West,Catholic,Catholic,,Justified,Male,Latino men,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,2.15388107299805,Non-white,,,,,,,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [1739]:
nat_df['IMPEACH18'].value_counts()

0    14548
1     2222
3     2203
2      248
Name: IMPEACH18, dtype: int64

In [1740]:
# no answer = 0 
# No = 1
# yes = 3
# omit = 2

### Question: 2017 total family income: 
Income 18 Gen: Original Question 
1. Under 30,000
2. 30,000-49,999
3. 50,000 - 99,999
4. 100,000 - 199,999
5. 200,000 or more 
    - V 4

In [1741]:
nat_df['INC100K'].value_counts()
#label 

                    14929
Under $100,000       2915
$100,000 or more     1377
Name: INC100K, dtype: int64

In [1742]:
label_converter(nat_df, 'INC100K').head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes
0,25480,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,0,,,Complete,,,,,23,0,0,0,0,1,1,2,White college degree,White women college graduates,0,0,0,1,0,0,0,0,0,0,2,0,3,0,,,,,,Gun policy,,No,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,White Democrat,Democratic women,Democrat,Democratic Women,White Democrat,Moderate,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,White,White,,,National questionnaire,White,Very important,Important,White,White,Whites are favored over minorities,West,Catholic,Catholic,Catholic,Justified,Female,White women,,,,"Cities 50,000-500,000","Cities over 50,000",English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,1.15386927127838,White Catholics,,,,,,,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,25481,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,Latino 30-44,Latino 30-44,0,,,Complete,,,,,40,0,0,0,0,3,2,2,Non white no college degree,Non-whites,0,0,0,1,0,0,0,0,0,0,1,0,3,0,,,,,,Health care,,Yes,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,Latino Democrat,Democratic men,Democrat,Democratic Men,Non-White Democrats,Liberal,,Liberal Democrats,Liberal Democrats,Other,Hispanic/Latino,,,National questionnaire,Hispanic/Latino,Somewhat important,Important,Non-white,Hispanic/Latino,Whites are favored over minorities,West,Catholic,Catholic,,Justified,Male,Latino men,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,2.15388107299805,Non-white,,,,,,,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [1743]:
nat_df['INC100K'].value_counts()

0    14929
2     2915
1     1377
Name: INC100K, dtype: int64

In [1744]:
# No answer = 0
# $100,000 or more = 2
# $100,000 or more = 1

### Same as last variable, INC30k

In [1745]:
nat_df['INC30K'].value_counts()

                   14929
$30,000 or more     3616
Under $30,000        676
Name: INC30K, dtype: int64

In [1746]:
label_converter(nat_df, "INC30K").head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes
0,25480,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,0,,,Complete,,,,,23,0,0,0,0,1,1,2,White college degree,White women college graduates,0,0,0,1,0,0,0,0,0,0,2,0,3,0,0,,,,,Gun policy,,No,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,White Democrat,Democratic women,Democrat,Democratic Women,White Democrat,Moderate,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,White,White,,,National questionnaire,White,Very important,Important,White,White,Whites are favored over minorities,West,Catholic,Catholic,Catholic,Justified,Female,White women,,,,"Cities 50,000-500,000","Cities over 50,000",English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,1.15386927127838,White Catholics,,,,,,,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,25481,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,Latino 30-44,Latino 30-44,0,,,Complete,,,,,40,0,0,0,0,3,2,2,Non white no college degree,Non-whites,0,0,0,1,0,0,0,0,0,0,1,0,3,0,0,,,,,Health care,,Yes,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,Latino Democrat,Democratic men,Democrat,Democratic Men,Non-White Democrats,Liberal,,Liberal Democrats,Liberal Democrats,Other,Hispanic/Latino,,,National questionnaire,Hispanic/Latino,Somewhat important,Important,Non-white,Hispanic/Latino,Whites are favored over minorities,West,Catholic,Catholic,,Justified,Male,Latino men,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,2.15388107299805,Non-white,,,,,,,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [1747]:
nat_df['INC30K'].value_counts()

0    14929
1     3616
2      676
Name: INC30K, dtype: int64

In [1748]:
# No answer = 0
# $30,000 or more = 1
# Under $30,000 or more = 2

### INC50K 

In [1749]:
nat_df['INC50K'].value_counts()

                   14929
$50,000 or more     2750
Under $50,000       1542
Name: INC50K, dtype: int64

In [1750]:
label_converter(nat_df, 'INC50K').head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes
0,25480,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,0,,,Complete,,,,,23,0,0,0,0,1,1,2,White college degree,White women college graduates,0,0,0,1,0,0,0,0,0,0,2,0,3,0,0,0,,,,Gun policy,,No,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,White Democrat,Democratic women,Democrat,Democratic Women,White Democrat,Moderate,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,White,White,,,National questionnaire,White,Very important,Important,White,White,Whites are favored over minorities,West,Catholic,Catholic,Catholic,Justified,Female,White women,,,,"Cities 50,000-500,000","Cities over 50,000",English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,1.15386927127838,White Catholics,,,,,,,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,25481,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,Latino 30-44,Latino 30-44,0,,,Complete,,,,,40,0,0,0,0,3,2,2,Non white no college degree,Non-whites,0,0,0,1,0,0,0,0,0,0,1,0,3,0,0,0,,,,Health care,,Yes,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,Latino Democrat,Democratic men,Democrat,Democratic Men,Non-White Democrats,Liberal,,Liberal Democrats,Liberal Democrats,Other,Hispanic/Latino,,,National questionnaire,Hispanic/Latino,Somewhat important,Important,Non-white,Hispanic/Latino,Whites are favored over minorities,West,Catholic,Catholic,,Justified,Male,Latino men,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,2.15388107299805,Non-white,,,,,,,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [1751]:
nat_df['INC50K'].value_counts()

0    14929
1     2750
2     1542
Name: INC50K, dtype: int64

In [1752]:
# No Answer = 0
# $50,000 or more = 1
# Under $50,000 = 2 

### Income3

In [1753]:
nat_df['INCOME3'].value_counts()

                      14929
Under $50,000          1542
$100,000               1377
$50,000 to $99,999     1373
Name: INCOME3, dtype: int64

In [1754]:
label_converter(nat_df, 'INCOME3').head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes
0,25480,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,0,,,Complete,,,,,23,0,0,0,0,1,1,2,White college degree,White women college graduates,0,0,0,1,0,0,0,0,0,0,2,0,3,0,0,0,0,,,Gun policy,,No,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,White Democrat,Democratic women,Democrat,Democratic Women,White Democrat,Moderate,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,White,White,,,National questionnaire,White,Very important,Important,White,White,Whites are favored over minorities,West,Catholic,Catholic,Catholic,Justified,Female,White women,,,,"Cities 50,000-500,000","Cities over 50,000",English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,1.15386927127838,White Catholics,,,,,,,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,25481,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,Latino 30-44,Latino 30-44,0,,,Complete,,,,,40,0,0,0,0,3,2,2,Non white no college degree,Non-whites,0,0,0,1,0,0,0,0,0,0,1,0,3,0,0,0,0,,,Health care,,Yes,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,Latino Democrat,Democratic men,Democrat,Democratic Men,Non-White Democrats,Liberal,,Liberal Democrats,Liberal Democrats,Other,Hispanic/Latino,,,National questionnaire,Hispanic/Latino,Somewhat important,Important,Non-white,Hispanic/Latino,Whites are favored over minorities,West,Catholic,Catholic,,Justified,Male,Latino men,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,2.15388107299805,Non-white,,,,,,,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [1755]:
nat_df['INCOME3'].value_counts()

0    14929
3     1542
1     1377
2     1373
Name: INCOME3, dtype: int64

In [1756]:
# No answer = 0 
# Under $50,000 = 3
# $100,000 = 1
# $50,000 to $99,999 = 2

### Original Question: INCOME18GEN

In [1757]:
nat_df['INCOME18GEN'].value_counts()

                     14929
$50,000-$99,999       1373
$100,000-$199,999     1046
$30,000-$49,999        866
Under $30,000          676
$200.000 or more       331
Name: INCOME18GEN, dtype: int64

In [1758]:
label_converter(nat_df, "INCOME18GEN").head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes
0,25480,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,0,,,Complete,,,,,23,0,0,0,0,1,1,2,White college degree,White women college graduates,0,0,0,1,0,0,0,0,0,0,2,0,3,0,0,0,0,0,,Gun policy,,No,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,White Democrat,Democratic women,Democrat,Democratic Women,White Democrat,Moderate,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,White,White,,,National questionnaire,White,Very important,Important,White,White,Whites are favored over minorities,West,Catholic,Catholic,Catholic,Justified,Female,White women,,,,"Cities 50,000-500,000","Cities over 50,000",English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,1.15386927127838,White Catholics,,,,,,,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,25481,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,Latino 30-44,Latino 30-44,0,,,Complete,,,,,40,0,0,0,0,3,2,2,Non white no college degree,Non-whites,0,0,0,1,0,0,0,0,0,0,1,0,3,0,0,0,0,0,,Health care,,Yes,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,Latino Democrat,Democratic men,Democrat,Democratic Men,Non-White Democrats,Liberal,,Liberal Democrats,Liberal Democrats,Other,Hispanic/Latino,,,National questionnaire,Hispanic/Latino,Somewhat important,Important,Non-white,Hispanic/Latino,Whites are favored over minorities,West,Catholic,Catholic,,Justified,Male,Latino men,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,2.15388107299805,Non-white,,,,,,,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [1759]:
nat_df['INCOME18GEN'].value_counts()

0    14929
4     1373
1     1046
3      866
5      676
2      331
Name: INCOME18GEN, dtype: int64

In [1760]:
# No answer = 0 
# $50,000-$99,999 = 4
# $100,000-$199,999 = 1
# $30,000-$49,999 = 3
# Under $30,000 = 5
# $200.000 or more = 2

### In your vote for U.S. House, how would you rate the importance of: 
##### Break 18 is the question for these answers 

In [1761]:
nat_df['ISSUE18'].value_counts()

               9921
Health care    4064
The economy    2051
Immigration    1774
Gun policy     1029
Omit            382
Name: ISSUE18, dtype: int64

In [1762]:
label_converter(nat_df, "ISSUE18").head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes
0,25480,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,0,,,Complete,,,,,23,0,0,0,0,1,1,2,White college degree,White women college graduates,0,0,0,1,0,0,0,0,0,0,2,0,3,0,0,0,0,0,,1,,No,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,White Democrat,Democratic women,Democrat,Democratic Women,White Democrat,Moderate,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,White,White,,,National questionnaire,White,Very important,Important,White,White,Whites are favored over minorities,West,Catholic,Catholic,Catholic,Justified,Female,White women,,,,"Cities 50,000-500,000","Cities over 50,000",English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,1.15386927127838,White Catholics,,,,,,,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,25481,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,Latino 30-44,Latino 30-44,0,,,Complete,,,,,40,0,0,0,0,3,2,2,Non white no college degree,Non-whites,0,0,0,1,0,0,0,0,0,0,1,0,3,0,0,0,0,0,,2,,Yes,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,Latino Democrat,Democratic men,Democrat,Democratic Men,Non-White Democrats,Liberal,,Liberal Democrats,Liberal Democrats,Other,Hispanic/Latino,,,National questionnaire,Hispanic/Latino,Somewhat important,Important,Non-white,Hispanic/Latino,Whites are favored over minorities,West,Catholic,Catholic,,Justified,Male,Latino men,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,2.15388107299805,Non-white,,,,,,,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [1763]:
nat_df['ISSUE18'].value_counts()

0    9921
2    4064
5    2051
3    1774
1    1029
4     382
Name: ISSUE18, dtype: int64

In [1764]:
# No answer = 0 
# Health care = 2
# The economy = 5
# Immigration = 3
# Gun policy = 1
# Omit = 4 

### How do you feel about Brett Kavanaugh's confirmation to the Supreme Court?: 
1. Support 
2. Oppose
    - V 1

In [1765]:
nat_df['KAVANAUGH18'].value_counts()

           14485
Oppose      2561
Support     1820
Omit         355
Name: KAVANAUGH18, dtype: int64

In [1766]:
label_converter(nat_df, "KAVANAUGH18").head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes
0,25480,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,0,,,Complete,,,,,23,0,0,0,0,1,1,2,White college degree,White women college graduates,0,0,0,1,0,0,0,0,0,0,2,0,3,0,0,0,0,0,,1,0,No,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,White Democrat,Democratic women,Democrat,Democratic Women,White Democrat,Moderate,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,White,White,,,National questionnaire,White,Very important,Important,White,White,Whites are favored over minorities,West,Catholic,Catholic,Catholic,Justified,Female,White women,,,,"Cities 50,000-500,000","Cities over 50,000",English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,1.15386927127838,White Catholics,,,,,,,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,25481,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,Latino 30-44,Latino 30-44,0,,,Complete,,,,,40,0,0,0,0,3,2,2,Non white no college degree,Non-whites,0,0,0,1,0,0,0,0,0,0,1,0,3,0,0,0,0,0,,2,0,Yes,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,Latino Democrat,Democratic men,Democrat,Democratic Men,Non-White Democrats,Liberal,,Liberal Democrats,Liberal Democrats,Other,Hispanic/Latino,,,National questionnaire,Hispanic/Latino,Somewhat important,Important,Non-white,Hispanic/Latino,Whites are favored over minorities,West,Catholic,Catholic,,Justified,Male,Latino men,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,2.15388107299805,Non-white,,,,,,,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [1767]:
nat_df['KAVANAUGH18'].value_counts()

0    14485
2     2561
3     1820
1      355
Name: KAVANAUGH18, dtype: int64

In [1768]:
# No answer = 0 
# Oppose = 2
# Support = 3
# Omit = 1

### Are you of Hispanic or Latino descent?: 
1. Yes
2. No 
    - V ALL 

In [1769]:
nat_df['LATINO'].value_counts()

No     15141
        2127
Yes     1953
Name: LATINO, dtype: int64

In [1770]:
label_converter(nat_df, 'LATINO').head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes
0,25480,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,0,,,Complete,,,,,23,0,0,0,0,1,1,2,White college degree,White women college graduates,0,0,0,1,0,0,0,0,0,0,2,0,3,0,0,0,0,0,,1,0,1,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,White Democrat,Democratic women,Democrat,Democratic Women,White Democrat,Moderate,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,White,White,,,National questionnaire,White,Very important,Important,White,White,Whites are favored over minorities,West,Catholic,Catholic,Catholic,Justified,Female,White women,,,,"Cities 50,000-500,000","Cities over 50,000",English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,1.15386927127838,White Catholics,,,,,,,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,25481,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,Latino 30-44,Latino 30-44,0,,,Complete,,,,,40,0,0,0,0,3,2,2,Non white no college degree,Non-whites,0,0,0,1,0,0,0,0,0,0,1,0,3,0,0,0,0,0,,2,0,2,,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,Latino Democrat,Democratic men,Democrat,Democratic Men,Non-White Democrats,Liberal,,Liberal Democrats,Liberal Democrats,Other,Hispanic/Latino,,,National questionnaire,Hispanic/Latino,Somewhat important,Important,Non-white,Hispanic/Latino,Whites are favored over minorities,West,Catholic,Catholic,,Justified,Male,Latino men,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,2.15388107299805,Non-white,,,,,,,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [1771]:
nat_df['LATINO'].value_counts()

1    15141
0     2127
2     1953
Name: LATINO, dtype: int64

In [1772]:
#No = 1
# No answer = 0
# Yes = 2 

### Are you gay, lesbian, bisexual or transgender?: 
1. Yes
2. No 
    - V-4

In [1773]:
nat_df['LGBT'].value_counts()

       14732
No      4213
Yes      276
Name: LGBT, dtype: int64

In [1774]:
label_converter(nat_df, 'LGBT').head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes
0,25480,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,0,,,Complete,,,,,23,0,0,0,0,1,1,2,White college degree,White women college graduates,0,0,0,1,0,0,0,0,0,0,2,0,3,0,0,0,0,0,,1,0,1,0,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,White Democrat,Democratic women,Democrat,Democratic Women,White Democrat,Moderate,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,White,White,,,National questionnaire,White,Very important,Important,White,White,Whites are favored over minorities,West,Catholic,Catholic,Catholic,Justified,Female,White women,,,,"Cities 50,000-500,000","Cities over 50,000",English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,1.15386927127838,White Catholics,,,,,,,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,25481,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,Latino 30-44,Latino 30-44,0,,,Complete,,,,,40,0,0,0,0,3,2,2,Non white no college degree,Non-whites,0,0,0,1,0,0,0,0,0,0,1,0,3,0,0,0,0,0,,2,0,2,0,,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,Latino Democrat,Democratic men,Democrat,Democratic Men,Non-White Democrats,Liberal,,Liberal Democrats,Liberal Democrats,Other,Hispanic/Latino,,,National questionnaire,Hispanic/Latino,Somewhat important,Important,Non-white,Hispanic/Latino,Whites are favored over minorities,West,Catholic,Catholic,,Justified,Male,Latino men,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,2.15388107299805,Non-white,,,,,,,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [1775]:
nat_df['LGBT'].value_counts()

0    14732
1     4213
2      276
Name: LGBT, dtype: int64

In [1776]:
# No answer = 0 
# No = 1
# Yes = 2 

### Are you currently married?: 
1. Yes 
2. No 
    - V-3

In [1777]:
nat_df['MARRIED'].value_counts()

       14514
Yes     2763
No      1944
Name: MARRIED, dtype: int64

In [1778]:
label_converter(nat_df, 'MARRIED').head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes
0,25480,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,0,,,Complete,,,,,23,0,0,0,0,1,1,2,White college degree,White women college graduates,0,0,0,1,0,0,0,0,0,0,2,0,3,0,0,0,0,0,,1,0,1,0,0,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,White Democrat,Democratic women,Democrat,Democratic Women,White Democrat,Moderate,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,White,White,,,National questionnaire,White,Very important,Important,White,White,Whites are favored over minorities,West,Catholic,Catholic,Catholic,Justified,Female,White women,,,,"Cities 50,000-500,000","Cities over 50,000",English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,1.15386927127838,White Catholics,,,,,,,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,25481,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,Latino 30-44,Latino 30-44,0,,,Complete,,,,,40,0,0,0,0,3,2,2,Non white no college degree,Non-whites,0,0,0,1,0,0,0,0,0,0,1,0,3,0,0,0,0,0,,2,0,2,0,0,Approve,,,18-44,18-64,,Democrat,Non-black Democrat,Latino Democrat,Democratic men,Democrat,Democratic Men,Non-White Democrats,Liberal,,Liberal Democrats,Liberal Democrats,Other,Hispanic/Latino,,,National questionnaire,Hispanic/Latino,Somewhat important,Important,Non-white,Hispanic/Latino,Whites are favored over minorities,West,Catholic,Catholic,,Justified,Male,Latino men,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,2.15388107299805,Non-white,,,,,,,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [1779]:
nat_df['MARRIED'].value_counts()

0    14514
2     2763
1     1944
Name: MARRIED, dtype: int64

In [1780]:
# No answer = 0 
# Yes = 2 
# No = 1

### How do you feel about Robert Mueller’s handling of the investigation into Russian interference in the 2016 election?: 
1. Approve 
2. Disapprove 
    - V 2 

In [1781]:
nat_df['MUELLERAPP'].value_counts()

              14548
Approve        2159
Disapprove     1918
Omit            596
Name: MUELLERAPP, dtype: int64

In [1782]:
label_converter(nat_df, 'MUELLERAPP').head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes
0,25480,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,0,,,Complete,,,,,23,0,0,0,0,1,1,2,White college degree,White women college graduates,0,0,0,1,0,0,0,0,0,0,2,0,3,0,0,0,0,0,,1,0,1,0,0,1,,,18-44,18-64,,Democrat,Non-black Democrat,White Democrat,Democratic women,Democrat,Democratic Women,White Democrat,Moderate,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,White,White,,,National questionnaire,White,Very important,Important,White,White,Whites are favored over minorities,West,Catholic,Catholic,Catholic,Justified,Female,White women,,,,"Cities 50,000-500,000","Cities over 50,000",English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,1.15386927127838,White Catholics,,,,,,,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,25481,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,Latino 30-44,Latino 30-44,0,,,Complete,,,,,40,0,0,0,0,3,2,2,Non white no college degree,Non-whites,0,0,0,1,0,0,0,0,0,0,1,0,3,0,0,0,0,0,,2,0,2,0,0,1,,,18-44,18-64,,Democrat,Non-black Democrat,Latino Democrat,Democratic men,Democrat,Democratic Men,Non-White Democrats,Liberal,,Liberal Democrats,Liberal Democrats,Other,Hispanic/Latino,,,National questionnaire,Hispanic/Latino,Somewhat important,Important,Non-white,Hispanic/Latino,Whites are favored over minorities,West,Catholic,Catholic,,Justified,Male,Latino men,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,2.15388107299805,Non-white,,,,,,,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [1783]:
nat_df['MUELLERAPP'].value_counts()

0    14548
1     2159
2     1918
3      596
Name: MUELLERAPP, dtype: int64

In [1784]:
# No answer = 0 
# Approve = 1
# Disapprove = 2
# Omit = 3 

### Do you think the condition of the nation's economy is:
1. Excellent 
2. Good 
3. Not so good 
4. Poor 
    - V-4

In [1785]:
nat_df['NEC'].value_counts()

               14635
Good            2381
Not so good     1207
Excellent        630
Poor             308
Omit              60
Name: NEC, dtype: int64

In [1786]:
label_converter(nat_df, 'NEC').head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes
0,25480,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,0,,,Complete,,,,,23,0,0,0,0,1,1,2,White college degree,White women college graduates,0,0,0,1,0,0,0,0,0,0,2,0,3,0,0,0,0,0,,1,0,1,0,0,1,0,,18-44,18-64,,Democrat,Non-black Democrat,White Democrat,Democratic women,Democrat,Democratic Women,White Democrat,Moderate,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,White,White,,,National questionnaire,White,Very important,Important,White,White,Whites are favored over minorities,West,Catholic,Catholic,Catholic,Justified,Female,White women,,,,"Cities 50,000-500,000","Cities over 50,000",English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,1.15386927127838,White Catholics,,,,,,,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,25481,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,Latino 30-44,Latino 30-44,0,,,Complete,,,,,40,0,0,0,0,3,2,2,Non white no college degree,Non-whites,0,0,0,1,0,0,0,0,0,0,1,0,3,0,0,0,0,0,,2,0,2,0,0,1,0,,18-44,18-64,,Democrat,Non-black Democrat,Latino Democrat,Democratic men,Democrat,Democratic Men,Non-White Democrats,Liberal,,Liberal Democrats,Liberal Democrats,Other,Hispanic/Latino,,,National questionnaire,Hispanic/Latino,Somewhat important,Important,Non-white,Hispanic/Latino,Whites are favored over minorities,West,Catholic,Catholic,,Justified,Male,Latino men,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,2.15388107299805,Non-white,,,,,,,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [1787]:
nat_df['NEC'].value_counts()

0    14635
2     2381
3     1207
1      630
5      308
4       60
Name: NEC, dtype: int64

In [1788]:
# No answer = 0 
# Good = 2
# Not so good = 3 
# Excellent = 1
# Poor = 5
# Omit = 4

### Feature interaction combined with variable NEC
1. Excellent or good
2. Not so good or poor
3. Omit
    - V-4

In [1789]:
nat_df['NEC2'].value_counts()

                       14635
Excellent or good       3011
Not so good or poor     1515
Omit                      60
Name: NEC2, dtype: int64

In [1790]:
label_converter(nat_df, 'NEC2').head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes
0,25480,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,0,,,Complete,,,,,23,0,0,0,0,1,1,2,White college degree,White women college graduates,0,0,0,1,0,0,0,0,0,0,2,0,3,0,0,0,0,0,,1,0,1,0,0,1,0,0,18-44,18-64,,Democrat,Non-black Democrat,White Democrat,Democratic women,Democrat,Democratic Women,White Democrat,Moderate,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,White,White,,,National questionnaire,White,Very important,Important,White,White,Whites are favored over minorities,West,Catholic,Catholic,Catholic,Justified,Female,White women,,,,"Cities 50,000-500,000","Cities over 50,000",English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,1.15386927127838,White Catholics,,,,,,,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,25481,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,Latino 30-44,Latino 30-44,0,,,Complete,,,,,40,0,0,0,0,3,2,2,Non white no college degree,Non-whites,0,0,0,1,0,0,0,0,0,0,1,0,3,0,0,0,0,0,,2,0,2,0,0,1,0,0,18-44,18-64,,Democrat,Non-black Democrat,Latino Democrat,Democratic men,Democrat,Democratic Men,Non-White Democrats,Liberal,,Liberal Democrats,Liberal Democrats,Other,Hispanic/Latino,,,National questionnaire,Hispanic/Latino,Somewhat important,Important,Non-white,Hispanic/Latino,Whites are favored over minorities,West,Catholic,Catholic,,Justified,Male,Latino men,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,2.15388107299805,Non-white,,,,,,,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [1791]:
nat_df['NEC2'].value_counts()

0    14635
1     3011
2     1515
3       60
Name: NEC2, dtype: int64

In [1792]:
# No answer = 0 
# Excellent or good = 1
# Not so good or poor = 2 
# Omit = 3 

### Related to BOYSGIRLS variable 
1. Women without childre
2. Men without children
3. Women with children
4. Men with children
    - V-3

In [1793]:
nat_df['PARENTS'].value_counts()

                          14656
Women without children     1669
Men without children       1371
Women with children         890
Men with children           635
Name: PARENTS, dtype: int64

In [1794]:
label_converter(nat_df, 'PARENTS').head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes
0,25480,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,0,,,Complete,,,,,23,0,0,0,0,1,1,2,White college degree,White women college graduates,0,0,0,1,0,0,0,0,0,0,2,0,3,0,0,0,0,0,,1,0,1,0,0,1,0,0,18-44,18-64,0,Democrat,Non-black Democrat,White Democrat,Democratic women,Democrat,Democratic Women,White Democrat,Moderate,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,White,White,,,National questionnaire,White,Very important,Important,White,White,Whites are favored over minorities,West,Catholic,Catholic,Catholic,Justified,Female,White women,,,,"Cities 50,000-500,000","Cities over 50,000",English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,1.15386927127838,White Catholics,,,,,,,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,25481,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,Latino 30-44,Latino 30-44,0,,,Complete,,,,,40,0,0,0,0,3,2,2,Non white no college degree,Non-whites,0,0,0,1,0,0,0,0,0,0,1,0,3,0,0,0,0,0,,2,0,2,0,0,1,0,0,18-44,18-64,0,Democrat,Non-black Democrat,Latino Democrat,Democratic men,Democrat,Democratic Men,Non-White Democrats,Liberal,,Liberal Democrats,Liberal Democrats,Other,Hispanic/Latino,,,National questionnaire,Hispanic/Latino,Somewhat important,Important,Non-white,Hispanic/Latino,Whites are favored over minorities,West,Catholic,Catholic,,Justified,Male,Latino men,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,2.15388107299805,Non-white,,,,,,,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [1795]:
nat_df['PARENTS'].value_counts()

0    14656
4     1669
2     1371
3      890
1      635
Name: PARENTS, dtype: int64

In [1796]:
# No answer = 0 
# Women without children = 4
# Men without children = 2
# Women with children = 3
# Men with childre = 1

### No matter how you voted today, do you usually think of yourself as a:
1. Democrat 
2. Republican 
3. Independent 
4. Something Else 
    - V all 

In [1797]:
nat_df['PARTY'].value_counts()

Democrat                      7746
Independent/Something else    5441
Republican                    5140
                               894
Name: PARTY, dtype: int64

In [1798]:
label_converter(nat_df, 'PARTY').head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes
0,25480,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,0,,,Complete,,,,,23,0,0,0,0,1,1,2,White college degree,White women college graduates,0,0,0,1,0,0,0,0,0,0,2,0,3,0,0,0,0,0,,1,0,1,0,0,1,0,0,18-44,18-64,0,1,Non-black Democrat,White Democrat,Democratic women,Democrat,Democratic Women,White Democrat,Moderate,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,White,White,,,National questionnaire,White,Very important,Important,White,White,Whites are favored over minorities,West,Catholic,Catholic,Catholic,Justified,Female,White women,,,,"Cities 50,000-500,000","Cities over 50,000",English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,1.15386927127838,White Catholics,,,,,,,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,25481,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,Latino 30-44,Latino 30-44,0,,,Complete,,,,,40,0,0,0,0,3,2,2,Non white no college degree,Non-whites,0,0,0,1,0,0,0,0,0,0,1,0,3,0,0,0,0,0,,2,0,2,0,0,1,0,0,18-44,18-64,0,1,Non-black Democrat,Latino Democrat,Democratic men,Democrat,Democratic Men,Non-White Democrats,Liberal,,Liberal Democrats,Liberal Democrats,Other,Hispanic/Latino,,,National questionnaire,Hispanic/Latino,Somewhat important,Important,Non-white,Hispanic/Latino,Whites are favored over minorities,West,Catholic,Catholic,,Justified,Male,Latino men,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,2.15388107299805,Non-white,,,,,,,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [1799]:
nat_df['PARTY'].value_counts()

1    7746
2    5441
3    5140
0     894
Name: PARTY, dtype: int64

In [1800]:
# Democrat = 1 
# Independent/ Something Else = 2
# Republican = 3
# No answer = 0 

### Original Party ID Question
1. Democrat 
2. Republican 
3. Independent 
4. Something Else 
    - V-all

In [1801]:
nat_df['PARTYID'].value_counts()

Democrat          7746
Republican        5140
Independent       4318
Something else    1123
                   894
Name: PARTYID, dtype: int64

In [1802]:
label_converter(nat_df, 'PARTYID').head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes
0,25480,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,0,,,Complete,,,,,23,0,0,0,0,1,1,2,White college degree,White women college graduates,0,0,0,1,0,0,0,0,0,0,2,0,3,0,0,0,0,0,,1,0,1,0,0,1,0,0,18-44,18-64,0,1,Non-black Democrat,White Democrat,Democratic women,1,Democratic Women,White Democrat,Moderate,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,White,White,,,National questionnaire,White,Very important,Important,White,White,Whites are favored over minorities,West,Catholic,Catholic,Catholic,Justified,Female,White women,,,,"Cities 50,000-500,000","Cities over 50,000",English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,1.15386927127838,White Catholics,,,,,,,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,25481,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,Latino 30-44,Latino 30-44,0,,,Complete,,,,,40,0,0,0,0,3,2,2,Non white no college degree,Non-whites,0,0,0,1,0,0,0,0,0,0,1,0,3,0,0,0,0,0,,2,0,2,0,0,1,0,0,18-44,18-64,0,1,Non-black Democrat,Latino Democrat,Democratic men,1,Democratic Men,Non-White Democrats,Liberal,,Liberal Democrats,Liberal Democrats,Other,Hispanic/Latino,,,National questionnaire,Hispanic/Latino,Somewhat important,Important,Non-white,Hispanic/Latino,Whites are favored over minorities,West,Catholic,Catholic,,Justified,Male,Latino men,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,2.15388107299805,Non-white,,,,,,,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [1803]:
nat_df['PARTYID'].value_counts()

1    7746
3    5140
2    4318
4    1123
0     894
Name: PARTYID, dtype: int64

In [1804]:
# Democrat = 1 
# Republican = 3 
# Independent = 2 
# Something else = 4 
# No answer = 0 

### On most political matters, do you consider yourself: 
1. Moderate 
2. Liberal 
3. Conservative 
    - V - All

In [1805]:
nat_df['PHIL3'].value_counts()

Moderate        7252
Liberal         5336
Conservative    5334
                1299
Name: PHIL3, dtype: int64

In [1806]:
label_converter(nat_df, 'PHIL3').head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes
0,25480,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,0,,,Complete,,,,,23,0,0,0,0,1,1,2,White college degree,White women college graduates,0,0,0,1,0,0,0,0,0,0,2,0,3,0,0,0,0,0,,1,0,1,0,0,1,0,0,18-44,18-64,0,1,Non-black Democrat,White Democrat,Democratic women,1,Democratic Women,White Democrat,3,,Moderate/Conservative Democrats,Mod/Conserv. Democrats,White,White,,,National questionnaire,White,Very important,Important,White,White,Whites are favored over minorities,West,Catholic,Catholic,Catholic,Justified,Female,White women,,,,"Cities 50,000-500,000","Cities over 50,000",English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,1.15386927127838,White Catholics,,,,,,,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,25481,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,Latino 30-44,Latino 30-44,0,,,Complete,,,,,40,0,0,0,0,3,2,2,Non white no college degree,Non-whites,0,0,0,1,0,0,0,0,0,0,1,0,3,0,0,0,0,0,,2,0,2,0,0,1,0,0,18-44,18-64,0,1,Non-black Democrat,Latino Democrat,Democratic men,1,Democratic Men,Non-White Democrats,2,,Liberal Democrats,Liberal Democrats,Other,Hispanic/Latino,,,National questionnaire,Hispanic/Latino,Somewhat important,Important,Non-white,Hispanic/Latino,Whites are favored over minorities,West,Catholic,Catholic,,Justified,Male,Latino men,,,,Suburbs,Suburbs,English,California,Other,,Telephone survey,,,,Before October,,,,,,Too tough,,Version 2,,,,2.15388107299805,Non-white,,,,,,,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [1807]:
nat_df['PHIL3'].value_counts()

3    7252
2    5336
1    5334
0    1299
Name: PHIL3, dtype: int64

In [1808]:
# Moderate = 3 
# Liberal = 2
# Conservative = 1 
# No Answer = 0 

In [1809]:
nat_df['PRECINCT'].value_counts()
# dummy 

       3400
16      266
50      258
51      228
56      224
       ... 
57        9
22        7
79        4
119       3
3         2
Name: PRECINCT, Length: 258, dtype: int64

In [1810]:
nat_df.loc[nat_df['PRECINCT'] == '16' ].head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PRECINCT,PTYIDEO,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes
4106,31419,30-59,30-39,18-44,18-49,30-44,30-39,Non-Black 30-44,White 30-44,White 30-44,0,,,Complete,,,,Call 1,6,0,0,0,0,1,1,2,White college degree,White men college graduates,0,0,0,3,0,0,0,0,0,0,1,0,1,0,0,0,0,0,,2,0,1,0,0,1,0,0,18-44,18-64,0,3,Non-black non-Democrat,All Republicans,Republican men,3,Republican Men,White Republican,3,16,Conservative Republicans,Mod./Liberal Republicans,White,White,,,National questionnaire,White,Somewhat important,Important,White,White,Whites are favored over minorities,Midwest,Catholic,Catholic,Catholic,Justified,Male,White men,,,,Suburbs,Suburbs,,Illinois,White suburban men,,Election day exit poll,In the last few days,In the last month,In the last week,,,,,,,Too tough,,Version 2,,,,.508285343647003,White Catholics,,,,,,,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
4130,31461,60+,65 or over,45+,50+,60+,65+,Non-Black 60+,White 60+,White 65+,0,,,Complete,,,,Call 1,6,0,0,0,0,1,1,2,White college degree,White women college graduates,0,1,3,0,2,1,1,0,0,0,0,0,0,2,1,1,2,4,"Over $50,000 and white",0,0,1,1,0,0,2,1,45+,65+,0,2,Non-black non-Democrat,White Independent,Independent women,2,Independent Women,White Independent,3,16,Moderate/Liberal Republicans,Moderate Independents,White,White,,,National questionnaire,White,,,White,White,,Midwest,,,,,Female,White women,,,,Suburbs,Suburbs,,Illinois,White suburban women,Hurt,Election day exit poll,,,,,Seriously off on the wrong track,No impact,,,,,Becoming more divided,Version 4,,,,.789462864398956,,No,,,,,,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [1811]:
nat_df = dummy_maker("PRECINCT", nat_df)

### Feature combination of PARTYID and PHIL 3 

In [1812]:
nat_df['PTYIDEO'].value_counts()
# Dummy

Moderate/Liberal Republicans       5296
Liberal Democrats                  3936
Moderate/Conservative Democrats    3529
Independent/Something else         3394
Conservative Republicans           1644
                                   1422
Name: PTYIDEO, dtype: int64

In [1813]:
nat_df = dummy_maker('PTYIDEO', nat_df)

In [1814]:
nat_df.head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PTYIDEO7,QRACE3,QRACEAI,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes,PRECINCT_2,PRECINCT_3,PRECINCT_4,PRECINCT_5,PRECINCT_6,PRECINCT_7,PRECINCT_8,PRECINCT_9,PRECINCT_10,PRECINCT_11,PRECINCT_12,PRECINCT_13,PRECINCT_14,PRECINCT_16,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_44,PRECINCT_45,PRECINCT_46,PRECINCT_47,PRECINCT_48,PRECINCT_49,PRECINCT_50,PRECINCT_51,PRECINCT_52,PRECINCT_53,PRECINCT_54,PRECINCT_55,PRECINCT_56,PRECINCT_57,PRECINCT_58,PRECINCT_59,PRECINCT_60,PRECINCT_61,PRECINCT_62,PRECINCT_63,PRECINCT_64,PRECINCT_65,PRECINCT_67,PRECINCT_68,PRECINCT_70,PRECINCT_71,PRECINCT_72,PRECINCT_73,PRECINCT_74,PRECINCT_75,PRECINCT_76,PRECINCT_77,PRECINCT_78,PRECINCT_79,PRECINCT_80,PRECINCT_81,PRECINCT_82,PRECINCT_83,PRECINCT_84,PRECINCT_85,PRECINCT_86,PRECINCT_87,PRECINCT_88,PRECINCT_89,PRECINCT_90,PRECINCT_91,PRECINCT_92,PRECINCT_93,PRECINCT_94,PRECINCT_95,PRECINCT_96,PRECINCT_97,PRECINCT_98,PRECINCT_99,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_130,PRECINCT_131,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_160,PRECINCT_234,PRECINCT_,PRECINCT_1,PRECINCT_10,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_11,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_12,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_13,PRECINCT_130,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_16,PRECINCT_160,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_234,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_3,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_4,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_4

### Question: Are you: (QRACE3)
1. White 
2. Other 
3. Black
    - V-All

In [1815]:
nat_df['QRACE3'].value_counts()
#dummy 

White    13207
Other     3085
Black     2587
           342
Name: QRACE3, dtype: int64

In [1816]:
nat_df = dummy_maker('QRACE3', nat_df)

### Original variable
#### Question: Are you: (QRACEAI)
1. White 
2. Black 
3. Hispanic/Latino 
4. Asian 
5. American Indian
6. Other 
    - V-All

In [1817]:
nat_df['QRACEAI'].value_counts()

White              13291
Black               2679
Hispanic/Latino     1458
                     784
Other                441
Asian                400
American Indian      168
Name: QRACEAI, dtype: int64

In [1818]:
nat_df = dummy_maker('QRACEAI', nat_df)

In [1819]:
nat_df.head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PTYIDEO7,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes,PRECINCT_2,PRECINCT_3,PRECINCT_4,PRECINCT_5,PRECINCT_6,PRECINCT_7,PRECINCT_8,PRECINCT_9,PRECINCT_10,PRECINCT_11,PRECINCT_12,PRECINCT_13,PRECINCT_14,PRECINCT_16,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_44,PRECINCT_45,PRECINCT_46,PRECINCT_47,PRECINCT_48,PRECINCT_49,PRECINCT_50,PRECINCT_51,PRECINCT_52,PRECINCT_53,PRECINCT_54,PRECINCT_55,PRECINCT_56,PRECINCT_57,PRECINCT_58,PRECINCT_59,PRECINCT_60,PRECINCT_61,PRECINCT_62,PRECINCT_63,PRECINCT_64,PRECINCT_65,PRECINCT_67,PRECINCT_68,PRECINCT_70,PRECINCT_71,PRECINCT_72,PRECINCT_73,PRECINCT_74,PRECINCT_75,PRECINCT_76,PRECINCT_77,PRECINCT_78,PRECINCT_79,PRECINCT_80,PRECINCT_81,PRECINCT_82,PRECINCT_83,PRECINCT_84,PRECINCT_85,PRECINCT_86,PRECINCT_87,PRECINCT_88,PRECINCT_89,PRECINCT_90,PRECINCT_91,PRECINCT_92,PRECINCT_93,PRECINCT_94,PRECINCT_95,PRECINCT_96,PRECINCT_97,PRECINCT_98,PRECINCT_99,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_130,PRECINCT_131,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_160,PRECINCT_234,PRECINCT_,PRECINCT_1,PRECINCT_10,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_11,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_12,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_13,PRECINCT_130,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_16,PRECINCT_160,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_234,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_3,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_4,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_44,PRECINCT_45,P

### How important is it that more racial and ethnic minorities be elected to public office?: 
1. Very Important 
2. Somewhat important 
3. Not too important 
4. Not at all important 
    - V - 2

In [1820]:
nat_df['RACE18'].value_counts()
#label 

                        14548
Very important           2263
Somewhat important       1228
Not at all important      518
Not too important         490
Omit                      174
Name: RACE18, dtype: int64

In [1821]:
label_converter(nat_df, 'RACE18').head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PTYIDEO7,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes,PRECINCT_2,PRECINCT_3,PRECINCT_4,PRECINCT_5,PRECINCT_6,PRECINCT_7,PRECINCT_8,PRECINCT_9,PRECINCT_10,PRECINCT_11,PRECINCT_12,PRECINCT_13,PRECINCT_14,PRECINCT_16,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_44,PRECINCT_45,PRECINCT_46,PRECINCT_47,PRECINCT_48,PRECINCT_49,PRECINCT_50,PRECINCT_51,PRECINCT_52,PRECINCT_53,PRECINCT_54,PRECINCT_55,PRECINCT_56,PRECINCT_57,PRECINCT_58,PRECINCT_59,PRECINCT_60,PRECINCT_61,PRECINCT_62,PRECINCT_63,PRECINCT_64,PRECINCT_65,PRECINCT_67,PRECINCT_68,PRECINCT_70,PRECINCT_71,PRECINCT_72,PRECINCT_73,PRECINCT_74,PRECINCT_75,PRECINCT_76,PRECINCT_77,PRECINCT_78,PRECINCT_79,PRECINCT_80,PRECINCT_81,PRECINCT_82,PRECINCT_83,PRECINCT_84,PRECINCT_85,PRECINCT_86,PRECINCT_87,PRECINCT_88,PRECINCT_89,PRECINCT_90,PRECINCT_91,PRECINCT_92,PRECINCT_93,PRECINCT_94,PRECINCT_95,PRECINCT_96,PRECINCT_97,PRECINCT_98,PRECINCT_99,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_130,PRECINCT_131,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_160,PRECINCT_234,PRECINCT_,PRECINCT_1,PRECINCT_10,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_11,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_12,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_13,PRECINCT_130,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_16,PRECINCT_160,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_234,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_3,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_4,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_44,PRECINCT_45,P

In [1822]:
nat_df['RACE18'].value_counts()


0    14548
5     2263
4     1228
1      518
2      490
3      174
Name: RACE18, dtype: int64

In [1823]:
# No answer = 0 
# Very important = 5
# Somewhat Important = 4 
# Not at all important = 1 
# Not too important = 2 
# Omit = 3 

### In general, how do you think things work in the U.S. today?: 
1. Whites are favored over minorities 
2. No group is favored 
3. Minorities are favored over whites 
4. Omit 
    - V -2 

In [1824]:
nat_df['RACEFACTOR'].value_counts()

                                      14548
Whites are favored over minorities     2308
No group is favored                    1351
Minorities are favored over whites      731
Omit                                    283
Name: RACEFACTOR, dtype: int64

In [1825]:
label_converter(nat_df, 'RACEFACTOR').head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PTYIDEO7,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REGION,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes,PRECINCT_2,PRECINCT_3,PRECINCT_4,PRECINCT_5,PRECINCT_6,PRECINCT_7,PRECINCT_8,PRECINCT_9,PRECINCT_10,PRECINCT_11,PRECINCT_12,PRECINCT_13,PRECINCT_14,PRECINCT_16,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_44,PRECINCT_45,PRECINCT_46,PRECINCT_47,PRECINCT_48,PRECINCT_49,PRECINCT_50,PRECINCT_51,PRECINCT_52,PRECINCT_53,PRECINCT_54,PRECINCT_55,PRECINCT_56,PRECINCT_57,PRECINCT_58,PRECINCT_59,PRECINCT_60,PRECINCT_61,PRECINCT_62,PRECINCT_63,PRECINCT_64,PRECINCT_65,PRECINCT_67,PRECINCT_68,PRECINCT_70,PRECINCT_71,PRECINCT_72,PRECINCT_73,PRECINCT_74,PRECINCT_75,PRECINCT_76,PRECINCT_77,PRECINCT_78,PRECINCT_79,PRECINCT_80,PRECINCT_81,PRECINCT_82,PRECINCT_83,PRECINCT_84,PRECINCT_85,PRECINCT_86,PRECINCT_87,PRECINCT_88,PRECINCT_89,PRECINCT_90,PRECINCT_91,PRECINCT_92,PRECINCT_93,PRECINCT_94,PRECINCT_95,PRECINCT_96,PRECINCT_97,PRECINCT_98,PRECINCT_99,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_130,PRECINCT_131,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_160,PRECINCT_234,PRECINCT_,PRECINCT_1,PRECINCT_10,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_11,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_12,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_13,PRECINCT_130,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_16,PRECINCT_160,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_234,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_3,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_4,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_44,PRECINCT_45,P

In [1826]:
nat_df['RACEFACTOR'].value_counts()

0    14548
4     2308
2     1351
1      731
3      283
Name: RACEFACTOR, dtype: int64

In [1827]:
# No answer = 0 
# Whites are favored over minorities = 4 
# No group is favored = 2
# Minorities are favored over whites = 1
# Omit = 3 

### Question: Can you please tell me your home zip code?

In [1828]:
nat_df['REGION'].value_counts()
#Dummy 

Midwest    5217
East       5060
South      4837
West       4107
Name: REGION, dtype: int64

In [1829]:
nat_df = dummy_maker("REGION", nat_df)
nat_df.head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PTYIDEO7,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes,PRECINCT_2,PRECINCT_3,PRECINCT_4,PRECINCT_5,PRECINCT_6,PRECINCT_7,PRECINCT_8,PRECINCT_9,PRECINCT_10,PRECINCT_11,PRECINCT_12,PRECINCT_13,PRECINCT_14,PRECINCT_16,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_44,PRECINCT_45,PRECINCT_46,PRECINCT_47,PRECINCT_48,PRECINCT_49,PRECINCT_50,PRECINCT_51,PRECINCT_52,PRECINCT_53,PRECINCT_54,PRECINCT_55,PRECINCT_56,PRECINCT_57,PRECINCT_58,PRECINCT_59,PRECINCT_60,PRECINCT_61,PRECINCT_62,PRECINCT_63,PRECINCT_64,PRECINCT_65,PRECINCT_67,PRECINCT_68,PRECINCT_70,PRECINCT_71,PRECINCT_72,PRECINCT_73,PRECINCT_74,PRECINCT_75,PRECINCT_76,PRECINCT_77,PRECINCT_78,PRECINCT_79,PRECINCT_80,PRECINCT_81,PRECINCT_82,PRECINCT_83,PRECINCT_84,PRECINCT_85,PRECINCT_86,PRECINCT_87,PRECINCT_88,PRECINCT_89,PRECINCT_90,PRECINCT_91,PRECINCT_92,PRECINCT_93,PRECINCT_94,PRECINCT_95,PRECINCT_96,PRECINCT_97,PRECINCT_98,PRECINCT_99,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_130,PRECINCT_131,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_160,PRECINCT_234,PRECINCT_,PRECINCT_1,PRECINCT_10,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_11,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_12,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_13,PRECINCT_130,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_16,PRECINCT_160,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_234,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_3,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_4,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_44,PRECINCT_45,PRECINCT

### Do you think the investigation into Russian interference in the 2016 election is mostly:
1. Justified 
2. Politically Motivated 
    - V-2

In [1830]:
nat_df['RUSSIA18'].value_counts()

                         14548
Justified                 2200
Politically motivated     2190
Omit                       283
Name: RUSSIA18, dtype: int64

In [1831]:
label_converter(nat_df, 'RUSSIA18').head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PTYIDEO7,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REL5,RELIGN18,RELWHITE,RUSSIA18,SEX,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes,PRECINCT_2,PRECINCT_3,PRECINCT_4,PRECINCT_5,PRECINCT_6,PRECINCT_7,PRECINCT_8,PRECINCT_9,PRECINCT_10,PRECINCT_11,PRECINCT_12,PRECINCT_13,PRECINCT_14,PRECINCT_16,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_44,PRECINCT_45,PRECINCT_46,PRECINCT_47,PRECINCT_48,PRECINCT_49,PRECINCT_50,PRECINCT_51,PRECINCT_52,PRECINCT_53,PRECINCT_54,PRECINCT_55,PRECINCT_56,PRECINCT_57,PRECINCT_58,PRECINCT_59,PRECINCT_60,PRECINCT_61,PRECINCT_62,PRECINCT_63,PRECINCT_64,PRECINCT_65,PRECINCT_67,PRECINCT_68,PRECINCT_70,PRECINCT_71,PRECINCT_72,PRECINCT_73,PRECINCT_74,PRECINCT_75,PRECINCT_76,PRECINCT_77,PRECINCT_78,PRECINCT_79,PRECINCT_80,PRECINCT_81,PRECINCT_82,PRECINCT_83,PRECINCT_84,PRECINCT_85,PRECINCT_86,PRECINCT_87,PRECINCT_88,PRECINCT_89,PRECINCT_90,PRECINCT_91,PRECINCT_92,PRECINCT_93,PRECINCT_94,PRECINCT_95,PRECINCT_96,PRECINCT_97,PRECINCT_98,PRECINCT_99,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_130,PRECINCT_131,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_160,PRECINCT_234,PRECINCT_,PRECINCT_1,PRECINCT_10,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_11,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_12,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_13,PRECINCT_130,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_16,PRECINCT_160,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_234,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_3,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_4,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_44,PRECINCT_45,PRECINCT

In [1832]:
nat_df['RUSSIA18'].value_counts()

0    14548
1     2200
3     2190
2      283
Name: RUSSIA18, dtype: int64

In [1833]:
# No answer = 0 
# Justified = 1 
# Politically motivated = 3 
# Omit = 2 

## Question: Are you: 
1. Male 
2. Female 
    - V-All 

In [1834]:
nat_df['SEX'].value_counts()

Female    10629
Male       8553
             39
Name: SEX, dtype: int64

In [1835]:
nat_df = dummy_maker('SEX', nat_df)
nat_df.head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PTYIDEO7,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REL5,RELIGN18,RELWHITE,RUSSIA18,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes,PRECINCT_2,PRECINCT_3,PRECINCT_4,PRECINCT_5,PRECINCT_6,PRECINCT_7,PRECINCT_8,PRECINCT_9,PRECINCT_10,PRECINCT_11,PRECINCT_12,PRECINCT_13,PRECINCT_14,PRECINCT_16,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_44,PRECINCT_45,PRECINCT_46,PRECINCT_47,PRECINCT_48,PRECINCT_49,PRECINCT_50,PRECINCT_51,PRECINCT_52,PRECINCT_53,PRECINCT_54,PRECINCT_55,PRECINCT_56,PRECINCT_57,PRECINCT_58,PRECINCT_59,PRECINCT_60,PRECINCT_61,PRECINCT_62,PRECINCT_63,PRECINCT_64,PRECINCT_65,PRECINCT_67,PRECINCT_68,PRECINCT_70,PRECINCT_71,PRECINCT_72,PRECINCT_73,PRECINCT_74,PRECINCT_75,PRECINCT_76,PRECINCT_77,PRECINCT_78,PRECINCT_79,PRECINCT_80,PRECINCT_81,PRECINCT_82,PRECINCT_83,PRECINCT_84,PRECINCT_85,PRECINCT_86,PRECINCT_87,PRECINCT_88,PRECINCT_89,PRECINCT_90,PRECINCT_91,PRECINCT_92,PRECINCT_93,PRECINCT_94,PRECINCT_95,PRECINCT_96,PRECINCT_97,PRECINCT_98,PRECINCT_99,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_130,PRECINCT_131,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_160,PRECINCT_234,PRECINCT_,PRECINCT_1,PRECINCT_10,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_11,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_12,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_13,PRECINCT_130,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_16,PRECINCT_160,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_234,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_3,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_4,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_44,PRECINCT_45,PRECINCT_47,

### How serious a problem is sexual harassment in this country today?: 
1. Very serious 
2. Somewhat serious 
3. Not too serious 
4. Not at all serious 
5. Omit 
    - V - 3

In [1836]:
nat_df['SEXUALHARASS'].value_counts()

                      14594
Very serious           2345
Somewhat serious       1608
Not too serious         473
Not at all serious      102
Omit                     99
Name: SEXUALHARASS, dtype: int64

In [1837]:
label_converter(nat_df, 'SEXUALHARASS').head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PTYIDEO7,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REL5,RELIGN18,RELWHITE,RUSSIA18,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes,PRECINCT_2,PRECINCT_3,PRECINCT_4,PRECINCT_5,PRECINCT_6,PRECINCT_7,PRECINCT_8,PRECINCT_9,PRECINCT_10,PRECINCT_11,PRECINCT_12,PRECINCT_13,PRECINCT_14,PRECINCT_16,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_44,PRECINCT_45,PRECINCT_46,PRECINCT_47,PRECINCT_48,PRECINCT_49,PRECINCT_50,PRECINCT_51,PRECINCT_52,PRECINCT_53,PRECINCT_54,PRECINCT_55,PRECINCT_56,PRECINCT_57,PRECINCT_58,PRECINCT_59,PRECINCT_60,PRECINCT_61,PRECINCT_62,PRECINCT_63,PRECINCT_64,PRECINCT_65,PRECINCT_67,PRECINCT_68,PRECINCT_70,PRECINCT_71,PRECINCT_72,PRECINCT_73,PRECINCT_74,PRECINCT_75,PRECINCT_76,PRECINCT_77,PRECINCT_78,PRECINCT_79,PRECINCT_80,PRECINCT_81,PRECINCT_82,PRECINCT_83,PRECINCT_84,PRECINCT_85,PRECINCT_86,PRECINCT_87,PRECINCT_88,PRECINCT_89,PRECINCT_90,PRECINCT_91,PRECINCT_92,PRECINCT_93,PRECINCT_94,PRECINCT_95,PRECINCT_96,PRECINCT_97,PRECINCT_98,PRECINCT_99,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_130,PRECINCT_131,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_160,PRECINCT_234,PRECINCT_,PRECINCT_1,PRECINCT_10,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_11,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_12,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_13,PRECINCT_130,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_16,PRECINCT_160,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_234,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_3,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_4,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_44,PRECINCT_45,PRECINCT_47,

In [1838]:
nat_df['SEXUALHARASS'].value_counts()

0    14594
5     2345
4     1608
2      473
1      102
3       99
Name: SEXUALHARASS, dtype: int64

In [1839]:
# No answer = 0 
# Very Serious = 5 
# Somewhat Serious = 4 
# Not too serious = 2 
# Not at all serious = 1
# Omit = 3 

### Less descriptive version of the sexual harassment question(not included)

In [1840]:
nat_df['SEXUALHARASSB'].value_counts()

               14594
Serious         3953
Not serious      575
Omit              99
Name: SEXUALHARASSB, dtype: int64

### A combination of married and sex 
    - V -3 

In [1841]:
nat_df['SEXWED'].value_counts()

                     14520
Married women         1449
Married men           1311
Non married women     1179
Non married men        762
Name: SEXWED, dtype: int64

In [1842]:
nat_df.loc[nat_df['SEXWED'] == "Married women"].tail(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PTYIDEO7,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REL5,RELIGN18,RELWHITE,RUSSIA18,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes,PRECINCT_2,PRECINCT_3,PRECINCT_4,PRECINCT_5,PRECINCT_6,PRECINCT_7,PRECINCT_8,PRECINCT_9,PRECINCT_10,PRECINCT_11,PRECINCT_12,PRECINCT_13,PRECINCT_14,PRECINCT_16,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_44,PRECINCT_45,PRECINCT_46,PRECINCT_47,PRECINCT_48,PRECINCT_49,PRECINCT_50,PRECINCT_51,PRECINCT_52,PRECINCT_53,PRECINCT_54,PRECINCT_55,PRECINCT_56,PRECINCT_57,PRECINCT_58,PRECINCT_59,PRECINCT_60,PRECINCT_61,PRECINCT_62,PRECINCT_63,PRECINCT_64,PRECINCT_65,PRECINCT_67,PRECINCT_68,PRECINCT_70,PRECINCT_71,PRECINCT_72,PRECINCT_73,PRECINCT_74,PRECINCT_75,PRECINCT_76,PRECINCT_77,PRECINCT_78,PRECINCT_79,PRECINCT_80,PRECINCT_81,PRECINCT_82,PRECINCT_83,PRECINCT_84,PRECINCT_85,PRECINCT_86,PRECINCT_87,PRECINCT_88,PRECINCT_89,PRECINCT_90,PRECINCT_91,PRECINCT_92,PRECINCT_93,PRECINCT_94,PRECINCT_95,PRECINCT_96,PRECINCT_97,PRECINCT_98,PRECINCT_99,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_130,PRECINCT_131,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_160,PRECINCT_234,PRECINCT_,PRECINCT_1,PRECINCT_10,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_11,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_12,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_13,PRECINCT_130,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_16,PRECINCT_160,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_234,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_3,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_4,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_44,PRECINCT_45,PRECINCT_47,

In [1843]:
dummy_maker('SEXWED', nat_df)
nat_df.head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PTYIDEO7,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REL5,RELIGN18,RELWHITE,RUSSIA18,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes,PRECINCT_2,PRECINCT_3,PRECINCT_4,PRECINCT_5,PRECINCT_6,PRECINCT_7,PRECINCT_8,PRECINCT_9,PRECINCT_10,PRECINCT_11,PRECINCT_12,PRECINCT_13,PRECINCT_14,PRECINCT_16,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_44,PRECINCT_45,PRECINCT_46,PRECINCT_47,PRECINCT_48,PRECINCT_49,PRECINCT_50,PRECINCT_51,PRECINCT_52,PRECINCT_53,PRECINCT_54,PRECINCT_55,PRECINCT_56,PRECINCT_57,PRECINCT_58,PRECINCT_59,PRECINCT_60,PRECINCT_61,PRECINCT_62,PRECINCT_63,PRECINCT_64,PRECINCT_65,PRECINCT_67,PRECINCT_68,PRECINCT_70,PRECINCT_71,PRECINCT_72,PRECINCT_73,PRECINCT_74,PRECINCT_75,PRECINCT_76,PRECINCT_77,PRECINCT_78,PRECINCT_79,PRECINCT_80,PRECINCT_81,PRECINCT_82,PRECINCT_83,PRECINCT_84,PRECINCT_85,PRECINCT_86,PRECINCT_87,PRECINCT_88,PRECINCT_89,PRECINCT_90,PRECINCT_91,PRECINCT_92,PRECINCT_93,PRECINCT_94,PRECINCT_95,PRECINCT_96,PRECINCT_97,PRECINCT_98,PRECINCT_99,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_130,PRECINCT_131,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_160,PRECINCT_234,PRECINCT_,PRECINCT_1,PRECINCT_10,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_11,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_12,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_13,PRECINCT_130,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_16,PRECINCT_160,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_234,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_3,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_4,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_44,PRECINCT_45,PRECINCT_47,

### All Versions: question related to zipcode 

In [1844]:
nat_df['SIZEPLAC'].value_counts()

Suburbs                  9452
Cities 50,000-500,000    4133
Rural                    3140
Cities over 500,000      2151
Cities 10,000-50,000      345
Name: SIZEPLAC, dtype: int64

In [1845]:
label_converter(nat_df, "SIZEPLAC").head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PTYIDEO7,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REL5,RELIGN18,RELWHITE,RUSSIA18,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes,PRECINCT_2,PRECINCT_3,PRECINCT_4,PRECINCT_5,PRECINCT_6,PRECINCT_7,PRECINCT_8,PRECINCT_9,PRECINCT_10,PRECINCT_11,PRECINCT_12,PRECINCT_13,PRECINCT_14,PRECINCT_16,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_44,PRECINCT_45,PRECINCT_46,PRECINCT_47,PRECINCT_48,PRECINCT_49,PRECINCT_50,PRECINCT_51,PRECINCT_52,PRECINCT_53,PRECINCT_54,PRECINCT_55,PRECINCT_56,PRECINCT_57,PRECINCT_58,PRECINCT_59,PRECINCT_60,PRECINCT_61,PRECINCT_62,PRECINCT_63,PRECINCT_64,PRECINCT_65,PRECINCT_67,PRECINCT_68,PRECINCT_70,PRECINCT_71,PRECINCT_72,PRECINCT_73,PRECINCT_74,PRECINCT_75,PRECINCT_76,PRECINCT_77,PRECINCT_78,PRECINCT_79,PRECINCT_80,PRECINCT_81,PRECINCT_82,PRECINCT_83,PRECINCT_84,PRECINCT_85,PRECINCT_86,PRECINCT_87,PRECINCT_88,PRECINCT_89,PRECINCT_90,PRECINCT_91,PRECINCT_92,PRECINCT_93,PRECINCT_94,PRECINCT_95,PRECINCT_96,PRECINCT_97,PRECINCT_98,PRECINCT_99,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_130,PRECINCT_131,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_160,PRECINCT_234,PRECINCT_,PRECINCT_1,PRECINCT_10,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_11,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_12,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_13,PRECINCT_130,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_16,PRECINCT_160,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_234,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_3,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_4,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_44,PRECINCT_45,PRECINCT_47,

In [1846]:
nat_df['SIZEPLAC'].value_counts()

4    9452
1    4133
3    3140
2    2151
0     345
Name: SIZEPLAC, dtype: int64

In [1847]:
# Suburbs = 4 
# Cities 50,000-500,000 = 1
# Rural = 3
# Cities over 500,000 = 2
# Cities 10,000-50,000 = 0 

In [1848]:
# omit Spanish Q

### What state do you live in 

In [1849]:
nat_df['STANUM'].value_counts()

California         2122
 New York          1137
Pennsylvania        982
 Illinois           842
Ohio                831
Florida             808
Texas               787
 Michigan           685
Wisconsin           632
Missouri            613
New Jersey          599
 Georgia            476
 Virginia           472
Minnesota           457
 Utah               420
Connecticut         352
Hawaii              349
 Nebraska           348
Indiana             332
Alabama             324
Maine               313
 Maryland           312
 Vermont            306
Arkansas            305
Delaware            303
Massachusetts       297
 North Carolina     295
 New Mexico         276
South Carolina      273
Tennessee           269
Wyoming             248
Rhode Island        239
Oklahoma            226
 Iowa               225
 Louisiana          220
Kansas              220
 Mississippi        192
Kentucky            190
Oregon              164
Colorado            127
New Hampshire       114
West Virginia   

In [1850]:
label_converter(nat_df, 'STANUM').head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PTYIDEO7,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REL5,RELIGN18,RELWHITE,RUSSIA18,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes,PRECINCT_2,PRECINCT_3,PRECINCT_4,PRECINCT_5,PRECINCT_6,PRECINCT_7,PRECINCT_8,PRECINCT_9,PRECINCT_10,PRECINCT_11,PRECINCT_12,PRECINCT_13,PRECINCT_14,PRECINCT_16,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_44,PRECINCT_45,PRECINCT_46,PRECINCT_47,PRECINCT_48,PRECINCT_49,PRECINCT_50,PRECINCT_51,PRECINCT_52,PRECINCT_53,PRECINCT_54,PRECINCT_55,PRECINCT_56,PRECINCT_57,PRECINCT_58,PRECINCT_59,PRECINCT_60,PRECINCT_61,PRECINCT_62,PRECINCT_63,PRECINCT_64,PRECINCT_65,PRECINCT_67,PRECINCT_68,PRECINCT_70,PRECINCT_71,PRECINCT_72,PRECINCT_73,PRECINCT_74,PRECINCT_75,PRECINCT_76,PRECINCT_77,PRECINCT_78,PRECINCT_79,PRECINCT_80,PRECINCT_81,PRECINCT_82,PRECINCT_83,PRECINCT_84,PRECINCT_85,PRECINCT_86,PRECINCT_87,PRECINCT_88,PRECINCT_89,PRECINCT_90,PRECINCT_91,PRECINCT_92,PRECINCT_93,PRECINCT_94,PRECINCT_95,PRECINCT_96,PRECINCT_97,PRECINCT_98,PRECINCT_99,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_130,PRECINCT_131,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_160,PRECINCT_234,PRECINCT_,PRECINCT_1,PRECINCT_10,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_11,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_12,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_13,PRECINCT_130,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_16,PRECINCT_160,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_234,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_3,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_4,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_44,PRECINCT_45,PRECINCT_47,

In [1851]:
nat_df['STANUM'].value_counts()

19    2122
10    1137
40     982
2      842
37     831
23     808
45     787
6      685
47     632
31     613
35     599
0      476
14     472
30     457
12     420
21     352
24     349
8      348
25     332
15     324
28     313
5      312
13     306
18     305
22     303
29     297
11     295
9      276
42     273
44     269
48     248
41     239
38     226
3      225
26     220
4      220
7      192
27     190
39     164
20     127
34     114
46     106
1       90
32      85
17      84
33      81
16      61
43      21
36      11
Name: STANUM, dtype: int64

In [1852]:
# California = 19
# New York = 10 
# Pennsylvania = 40 
# Illinois = 2
# Ohio = 37
# Florida = 23 
# Texas = 45
# Michigan = 6  
# Wisconsin = 47
# Missouri = 31
# New Jersey = 35
# Georgia = 0
# Virginia = 14
# Minnesota = 30
# Utah = 12
# Connecticut = 21
# Hawaii = 24
# Nebraska = 8
# Indiana = 25
# Alabama = 15
# Maine = 28
# Maryland = 5 
# Vermont = 13 
# Arkansas = 18 
# Delaware = 22 
# Massachusetts = 29
# North Carolina = 11
# New Mexico = 9
# South Carolina = 42
# Tennessee = 44 
# Wyoming = 48 
# Rhode Island = 41 
# Oklahoma = 38 
# Iowa = 3 
# Louisiana = 26 
# Kansas = 4 
# Mississippi = 7 
# Kentucky = 27 
# Oregon = 39 
# Colorado = 20 
# New Hampshire = 34 
# West Virginia = 46 
# Idaho = 1 
# Montana = 32 
# Arizona = 17
# Nevada = 33 
# Alaska = 16 
# South Dakota = 43 
# North Dakota = 11 


### Question: How have the tax laws passed last year affected your personal finances?: 
1. Helped 
2. Hurt 
3. No impact 
    - V-4 

In [1853]:
nat_df["TAXCUTS18"].value_counts()

             14635
No impact     2026
Hurt          1235
Helped        1149
Omit           176
Name: TAXCUTS18, dtype: int64

In [1854]:
label_converter(nat_df, 'TAXCUTS18').head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PTYIDEO7,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REL5,RELIGN18,RELWHITE,RUSSIA18,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes,PRECINCT_2,PRECINCT_3,PRECINCT_4,PRECINCT_5,PRECINCT_6,PRECINCT_7,PRECINCT_8,PRECINCT_9,PRECINCT_10,PRECINCT_11,PRECINCT_12,PRECINCT_13,PRECINCT_14,PRECINCT_16,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_44,PRECINCT_45,PRECINCT_46,PRECINCT_47,PRECINCT_48,PRECINCT_49,PRECINCT_50,PRECINCT_51,PRECINCT_52,PRECINCT_53,PRECINCT_54,PRECINCT_55,PRECINCT_56,PRECINCT_57,PRECINCT_58,PRECINCT_59,PRECINCT_60,PRECINCT_61,PRECINCT_62,PRECINCT_63,PRECINCT_64,PRECINCT_65,PRECINCT_67,PRECINCT_68,PRECINCT_70,PRECINCT_71,PRECINCT_72,PRECINCT_73,PRECINCT_74,PRECINCT_75,PRECINCT_76,PRECINCT_77,PRECINCT_78,PRECINCT_79,PRECINCT_80,PRECINCT_81,PRECINCT_82,PRECINCT_83,PRECINCT_84,PRECINCT_85,PRECINCT_86,PRECINCT_87,PRECINCT_88,PRECINCT_89,PRECINCT_90,PRECINCT_91,PRECINCT_92,PRECINCT_93,PRECINCT_94,PRECINCT_95,PRECINCT_96,PRECINCT_97,PRECINCT_98,PRECINCT_99,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_130,PRECINCT_131,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_160,PRECINCT_234,PRECINCT_,PRECINCT_1,PRECINCT_10,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_11,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_12,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_13,PRECINCT_130,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_16,PRECINCT_160,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_234,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_3,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_4,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_44,PRECINCT_45,PRECINCT_47,

In [1855]:
nat_df["TAXCUTS18"].value_counts()

0    14635
3     2026
2     1235
1     1149
4      176
Name: TAXCUTS18, dtype: int64

In [1856]:
# No answer = 0 
# No Impact  = 3 
# Hurt = 2 
# Helped = 1
# Omit = 4 

In [1857]:
# Omit Telepoll, TIME18HOU_1, TIME18HOUWEEK_1, TIME18HOU_2 

### Do you think things in this country today are:
v-4 
1. Generally going in the right direction
2. Seriously off on the wrong track

In [1858]:
nat_df["TRACK"].value_counts()

                                          14635
Seriously off on the wrong track           2730
Generally going in the right direction     1686
Omit                                        170
Name: TRACK, dtype: int64

In [1859]:
label_converter(nat_df, 'TRACK').head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PTYIDEO7,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REL5,RELIGN18,RELWHITE,RUSSIA18,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes,PRECINCT_2,PRECINCT_3,PRECINCT_4,PRECINCT_5,PRECINCT_6,PRECINCT_7,PRECINCT_8,PRECINCT_9,PRECINCT_10,PRECINCT_11,PRECINCT_12,PRECINCT_13,PRECINCT_14,PRECINCT_16,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_44,PRECINCT_45,PRECINCT_46,PRECINCT_47,PRECINCT_48,PRECINCT_49,PRECINCT_50,PRECINCT_51,PRECINCT_52,PRECINCT_53,PRECINCT_54,PRECINCT_55,PRECINCT_56,PRECINCT_57,PRECINCT_58,PRECINCT_59,PRECINCT_60,PRECINCT_61,PRECINCT_62,PRECINCT_63,PRECINCT_64,PRECINCT_65,PRECINCT_67,PRECINCT_68,PRECINCT_70,PRECINCT_71,PRECINCT_72,PRECINCT_73,PRECINCT_74,PRECINCT_75,PRECINCT_76,PRECINCT_77,PRECINCT_78,PRECINCT_79,PRECINCT_80,PRECINCT_81,PRECINCT_82,PRECINCT_83,PRECINCT_84,PRECINCT_85,PRECINCT_86,PRECINCT_87,PRECINCT_88,PRECINCT_89,PRECINCT_90,PRECINCT_91,PRECINCT_92,PRECINCT_93,PRECINCT_94,PRECINCT_95,PRECINCT_96,PRECINCT_97,PRECINCT_98,PRECINCT_99,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_130,PRECINCT_131,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_160,PRECINCT_234,PRECINCT_,PRECINCT_1,PRECINCT_10,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_11,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_12,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_13,PRECINCT_130,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_16,PRECINCT_160,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_234,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_3,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_4,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_44,PRECINCT_45,PRECINCT_47,

In [1860]:
nat_df["TRACK"].value_counts()

0    14635
3     2730
1     1686
2      170
Name: TRACK, dtype: int64

In [1861]:
# No answer = 0 
# Seriously off on the wrong track = 3 
# Generally going in the right direction = 1 
# Omit = 2 

###  How have Donald Trump's trade policies affected the economy in your area?: 
v-4 
1. Helped 
2. Hurt 
3. No Impact 

In [1862]:
nat_df["TRADE18"].value_counts()

             14635
Hurt          1613
No impact     1610
Helped        1080
Omit           283
Name: TRADE18, dtype: int64

In [1863]:
label_converter(nat_df, 'TRADE18').head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PTYIDEO7,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REL5,RELIGN18,RELWHITE,RUSSIA18,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes,PRECINCT_2,PRECINCT_3,PRECINCT_4,PRECINCT_5,PRECINCT_6,PRECINCT_7,PRECINCT_8,PRECINCT_9,PRECINCT_10,PRECINCT_11,PRECINCT_12,PRECINCT_13,PRECINCT_14,PRECINCT_16,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_44,PRECINCT_45,PRECINCT_46,PRECINCT_47,PRECINCT_48,PRECINCT_49,PRECINCT_50,PRECINCT_51,PRECINCT_52,PRECINCT_53,PRECINCT_54,PRECINCT_55,PRECINCT_56,PRECINCT_57,PRECINCT_58,PRECINCT_59,PRECINCT_60,PRECINCT_61,PRECINCT_62,PRECINCT_63,PRECINCT_64,PRECINCT_65,PRECINCT_67,PRECINCT_68,PRECINCT_70,PRECINCT_71,PRECINCT_72,PRECINCT_73,PRECINCT_74,PRECINCT_75,PRECINCT_76,PRECINCT_77,PRECINCT_78,PRECINCT_79,PRECINCT_80,PRECINCT_81,PRECINCT_82,PRECINCT_83,PRECINCT_84,PRECINCT_85,PRECINCT_86,PRECINCT_87,PRECINCT_88,PRECINCT_89,PRECINCT_90,PRECINCT_91,PRECINCT_92,PRECINCT_93,PRECINCT_94,PRECINCT_95,PRECINCT_96,PRECINCT_97,PRECINCT_98,PRECINCT_99,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_130,PRECINCT_131,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_160,PRECINCT_234,PRECINCT_,PRECINCT_1,PRECINCT_10,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_11,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_12,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_13,PRECINCT_130,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_16,PRECINCT_160,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_234,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_3,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_4,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_44,PRECINCT_45,PRECINCT_47,

In [1864]:
nat_df["TRADE18"].value_counts()

0    14635
2     1613
3     1610
1     1080
4      283
Name: TRADE18, dtype: int64

In [1865]:
# No answer = 0 
# Hurt = 2 
# No impact = 3 
# Helped = 1 
# Omit = 4 

### How do you feel about the way Donald Trump is handling his job as president?: 
TRUMP4 
1. Strongly or somewhat approve
2. Somewhat Approve 
3. Somewhat Disapprove 
4. Strongly Disapprove 
    - V-1
    - V-3

In [1866]:
nat_df["TRUMP4"].value_counts()

                       9858
Strongly disapprove    4857
Strongly approve       2322
Somewhat approve       1226
Somewhat disapprove     854
Omit                    104
Name: TRUMP4, dtype: int64

In [1867]:
label_converter(nat_df, 'TRUMP4').head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PTYIDEO7,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REL5,RELIGN18,RELWHITE,RUSSIA18,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes,PRECINCT_2,PRECINCT_3,PRECINCT_4,PRECINCT_5,PRECINCT_6,PRECINCT_7,PRECINCT_8,PRECINCT_9,PRECINCT_10,PRECINCT_11,PRECINCT_12,PRECINCT_13,PRECINCT_14,PRECINCT_16,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_44,PRECINCT_45,PRECINCT_46,PRECINCT_47,PRECINCT_48,PRECINCT_49,PRECINCT_50,PRECINCT_51,PRECINCT_52,PRECINCT_53,PRECINCT_54,PRECINCT_55,PRECINCT_56,PRECINCT_57,PRECINCT_58,PRECINCT_59,PRECINCT_60,PRECINCT_61,PRECINCT_62,PRECINCT_63,PRECINCT_64,PRECINCT_65,PRECINCT_67,PRECINCT_68,PRECINCT_70,PRECINCT_71,PRECINCT_72,PRECINCT_73,PRECINCT_74,PRECINCT_75,PRECINCT_76,PRECINCT_77,PRECINCT_78,PRECINCT_79,PRECINCT_80,PRECINCT_81,PRECINCT_82,PRECINCT_83,PRECINCT_84,PRECINCT_85,PRECINCT_86,PRECINCT_87,PRECINCT_88,PRECINCT_89,PRECINCT_90,PRECINCT_91,PRECINCT_92,PRECINCT_93,PRECINCT_94,PRECINCT_95,PRECINCT_96,PRECINCT_97,PRECINCT_98,PRECINCT_99,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_130,PRECINCT_131,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_160,PRECINCT_234,PRECINCT_,PRECINCT_1,PRECINCT_10,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_11,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_12,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_13,PRECINCT_130,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_16,PRECINCT_160,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_234,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_3,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_4,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_44,PRECINCT_45,PRECINCT_47,

In [1868]:
# remove TRUMP2 

In [1869]:
nat_df["TRUMP4"].value_counts()

0    9858
5    4857
4    2322
2    1226
3     854
1     104
Name: TRUMP4, dtype: int64

In [1870]:
# No answer = 0 
# Strongly Disapprove = 5 
# Strongly Approve = 4 
# Somewhat approve = 2
# Somewhat disapprove = 3 
# Omit = 1 

### Is Donald Trump's approach to foreign policy making:

1. The U.S. more safe
2. The U.S. less safe
3. No difference
    - V-1 

In [1871]:
nat_df["TRUMPFP18"].value_counts()

                      14485
The U.S. less safe     2483
The U.S. more safe     1528
No difference           590
Omit                    135
Name: TRUMPFP18, dtype: int64

In [1872]:
label_converter(nat_df, 'TRUMPFP18').head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PTYIDEO7,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REL5,RELIGN18,RELWHITE,RUSSIA18,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes,PRECINCT_2,PRECINCT_3,PRECINCT_4,PRECINCT_5,PRECINCT_6,PRECINCT_7,PRECINCT_8,PRECINCT_9,PRECINCT_10,PRECINCT_11,PRECINCT_12,PRECINCT_13,PRECINCT_14,PRECINCT_16,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_44,PRECINCT_45,PRECINCT_46,PRECINCT_47,PRECINCT_48,PRECINCT_49,PRECINCT_50,PRECINCT_51,PRECINCT_52,PRECINCT_53,PRECINCT_54,PRECINCT_55,PRECINCT_56,PRECINCT_57,PRECINCT_58,PRECINCT_59,PRECINCT_60,PRECINCT_61,PRECINCT_62,PRECINCT_63,PRECINCT_64,PRECINCT_65,PRECINCT_67,PRECINCT_68,PRECINCT_70,PRECINCT_71,PRECINCT_72,PRECINCT_73,PRECINCT_74,PRECINCT_75,PRECINCT_76,PRECINCT_77,PRECINCT_78,PRECINCT_79,PRECINCT_80,PRECINCT_81,PRECINCT_82,PRECINCT_83,PRECINCT_84,PRECINCT_85,PRECINCT_86,PRECINCT_87,PRECINCT_88,PRECINCT_89,PRECINCT_90,PRECINCT_91,PRECINCT_92,PRECINCT_93,PRECINCT_94,PRECINCT_95,PRECINCT_96,PRECINCT_97,PRECINCT_98,PRECINCT_99,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_130,PRECINCT_131,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_160,PRECINCT_234,PRECINCT_,PRECINCT_1,PRECINCT_10,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_11,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_12,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_13,PRECINCT_130,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_16,PRECINCT_160,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_234,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_3,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_4,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_44,PRECINCT_45,PRECINCT_47,

In [1873]:
nat_df["TRUMPFP18"].value_counts()

0    14485
3     2483
4     1528
1      590
2      135
Name: TRUMPFP18, dtype: int64

In [1874]:
# No answer = 0 
# The U.S. less safe = 3 
# The U.S. more safe = 4 
# No difference = 590 
# Omit 

### Are Donald Trump's immigration policies:
1. Too Tough 
2. Not Tough Enough 
3. About Right
    - v-2

In [1875]:
nat_df["TRUMPIMM"].value_counts()

                    14548
Too tough            2483
About right          1312
Not tough enough      663
Omit                  215
Name: TRUMPIMM, dtype: int64

In [1876]:
label_converter(nat_df, "TRUMPIMM").head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PTYIDEO7,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REL5,RELIGN18,RELWHITE,RUSSIA18,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes,PRECINCT_2,PRECINCT_3,PRECINCT_4,PRECINCT_5,PRECINCT_6,PRECINCT_7,PRECINCT_8,PRECINCT_9,PRECINCT_10,PRECINCT_11,PRECINCT_12,PRECINCT_13,PRECINCT_14,PRECINCT_16,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_44,PRECINCT_45,PRECINCT_46,PRECINCT_47,PRECINCT_48,PRECINCT_49,PRECINCT_50,PRECINCT_51,PRECINCT_52,PRECINCT_53,PRECINCT_54,PRECINCT_55,PRECINCT_56,PRECINCT_57,PRECINCT_58,PRECINCT_59,PRECINCT_60,PRECINCT_61,PRECINCT_62,PRECINCT_63,PRECINCT_64,PRECINCT_65,PRECINCT_67,PRECINCT_68,PRECINCT_70,PRECINCT_71,PRECINCT_72,PRECINCT_73,PRECINCT_74,PRECINCT_75,PRECINCT_76,PRECINCT_77,PRECINCT_78,PRECINCT_79,PRECINCT_80,PRECINCT_81,PRECINCT_82,PRECINCT_83,PRECINCT_84,PRECINCT_85,PRECINCT_86,PRECINCT_87,PRECINCT_88,PRECINCT_89,PRECINCT_90,PRECINCT_91,PRECINCT_92,PRECINCT_93,PRECINCT_94,PRECINCT_95,PRECINCT_96,PRECINCT_97,PRECINCT_98,PRECINCT_99,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_130,PRECINCT_131,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_160,PRECINCT_234,PRECINCT_,PRECINCT_1,PRECINCT_10,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_11,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_12,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_13,PRECINCT_130,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_16,PRECINCT_160,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_234,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_3,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_4,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_44,PRECINCT_45,PRECINCT_47,

In [1877]:
nat_df["TRUMPIMM"].value_counts()

0    14548
4     2483
1     1312
2      663
3      215
Name: TRUMPIMM, dtype: int64

In [1878]:
# No answer = 0 
# Too Tough = 4 
# About Right = 1 
# Not tough enough = 2 
# Omit = 3

### Question:  Politically, do you think Americans are:
1. Becoming more united 
2. Becoming more divided 
3. Staying about the same
    - v-4 

In [1879]:
nat_df["UNITED18"].value_counts()

                          14635
Becoming more divided      3590
Staying about the same      535
Becoming more united        359
Omit                        102
Name: UNITED18, dtype: int64

In [1880]:
label_converter(nat_df,"UNITED18").head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PTYIDEO7,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REL5,RELIGN18,RELWHITE,RUSSIA18,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes,PRECINCT_2,PRECINCT_3,PRECINCT_4,PRECINCT_5,PRECINCT_6,PRECINCT_7,PRECINCT_8,PRECINCT_9,PRECINCT_10,PRECINCT_11,PRECINCT_12,PRECINCT_13,PRECINCT_14,PRECINCT_16,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_44,PRECINCT_45,PRECINCT_46,PRECINCT_47,PRECINCT_48,PRECINCT_49,PRECINCT_50,PRECINCT_51,PRECINCT_52,PRECINCT_53,PRECINCT_54,PRECINCT_55,PRECINCT_56,PRECINCT_57,PRECINCT_58,PRECINCT_59,PRECINCT_60,PRECINCT_61,PRECINCT_62,PRECINCT_63,PRECINCT_64,PRECINCT_65,PRECINCT_67,PRECINCT_68,PRECINCT_70,PRECINCT_71,PRECINCT_72,PRECINCT_73,PRECINCT_74,PRECINCT_75,PRECINCT_76,PRECINCT_77,PRECINCT_78,PRECINCT_79,PRECINCT_80,PRECINCT_81,PRECINCT_82,PRECINCT_83,PRECINCT_84,PRECINCT_85,PRECINCT_86,PRECINCT_87,PRECINCT_88,PRECINCT_89,PRECINCT_90,PRECINCT_91,PRECINCT_92,PRECINCT_93,PRECINCT_94,PRECINCT_95,PRECINCT_96,PRECINCT_97,PRECINCT_98,PRECINCT_99,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_130,PRECINCT_131,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_160,PRECINCT_234,PRECINCT_,PRECINCT_1,PRECINCT_10,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_11,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_12,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_13,PRECINCT_130,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_16,PRECINCT_160,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_234,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_3,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_4,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_44,PRECINCT_45,PRECINCT_47,

In [1881]:
nat_df["UNITED18"].value_counts()

0    14635
1     3590
4      535
2      359
3      102
Name: UNITED18, dtype: int64

In [1882]:
# No answer = 0 
# Becoming more divided = 1 
# Staying about the same = 4 
# Becoming more united = 2 
# Omit = 3 

###  In the 2016 election for president, did you vote for:

1. Hillary Clinton 
2. Donald Trump 
3. Other
4. Didn't Vote 
    - V-1 

In [1883]:
nat_df["VOTE2016"].value_counts()

                   14485
Hillary Clinton     2290
Donald Trump        1618
Other                418
Didn't vote          323
Omit                  87
Name: VOTE2016, dtype: int64

In [1884]:
label_converter(nat_df,'VOTE2016').head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PTYIDEO7,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REL5,RELIGN18,RELWHITE,RUSSIA18,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes,PRECINCT_2,PRECINCT_3,PRECINCT_4,PRECINCT_5,PRECINCT_6,PRECINCT_7,PRECINCT_8,PRECINCT_9,PRECINCT_10,PRECINCT_11,PRECINCT_12,PRECINCT_13,PRECINCT_14,PRECINCT_16,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_44,PRECINCT_45,PRECINCT_46,PRECINCT_47,PRECINCT_48,PRECINCT_49,PRECINCT_50,PRECINCT_51,PRECINCT_52,PRECINCT_53,PRECINCT_54,PRECINCT_55,PRECINCT_56,PRECINCT_57,PRECINCT_58,PRECINCT_59,PRECINCT_60,PRECINCT_61,PRECINCT_62,PRECINCT_63,PRECINCT_64,PRECINCT_65,PRECINCT_67,PRECINCT_68,PRECINCT_70,PRECINCT_71,PRECINCT_72,PRECINCT_73,PRECINCT_74,PRECINCT_75,PRECINCT_76,PRECINCT_77,PRECINCT_78,PRECINCT_79,PRECINCT_80,PRECINCT_81,PRECINCT_82,PRECINCT_83,PRECINCT_84,PRECINCT_85,PRECINCT_86,PRECINCT_87,PRECINCT_88,PRECINCT_89,PRECINCT_90,PRECINCT_91,PRECINCT_92,PRECINCT_93,PRECINCT_94,PRECINCT_95,PRECINCT_96,PRECINCT_97,PRECINCT_98,PRECINCT_99,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_130,PRECINCT_131,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_160,PRECINCT_234,PRECINCT_,PRECINCT_1,PRECINCT_10,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_11,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_12,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_13,PRECINCT_130,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_16,PRECINCT_160,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_234,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_3,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_4,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_44,PRECINCT_45,PRECINCT_47,

In [1885]:
nat_df["VOTE2016"].value_counts()

0    14485
3     2290
2     1618
5      418
1      323
4       87
Name: VOTE2016, dtype: int64

In [1886]:
# No answer = 0
# Hillary Clinton = 3 
# Donald Trump = 2 
# Other = 5 
# Didn't Vote = 1 
# Omit = 4 

### How important is it that more women be elected to public office?: 
1. Very Important
2. Somewhat Important 
3. Not too important 
4. Not important at all 
    - V -3

In [1887]:
nat_df["WOMEN"].value_counts()

                        14594
Very important           2288
Somewhat important       1435
Not too important         544
Not at all important      287
Omit                       73
Name: WOMEN, dtype: int64

In [1382]:
label_converter(nat_df, 'WOMEN').head(2)

,ID,AGE3,AGE8,AGE45,AGE49,AGE60,AGE65,AGEBLACK,AGEBYRACE,AGEBYRACE08,ATTEND16,ATTEND16B,ATTREL,BACKSIDE,BREAK18,BREAK18A,BREAK18B,CALL,CDNUM,CHILD18,CHILDBOY,CHILDGIRL,CONTROLHOUSE,EDUC18,EDUCCOLL,EDUCHS,EDUCWHITE,EDUCWHITEBYSEX,ELECTSECFOREIGN18,FAVDEM2,FAVPARTY,FAVPELOSI,FAVREP2,FINSIT2,FORTRUMPH,FTVOTER18,GUN16,GUNOWNER12,HEALTHCARE18,HEALTHPRE18,IMPEACH18,INC100K,INC30K,INC50K,INCOME3,INCOME18GEN,INCWHITE,ISSUE18,KAVANAUGH18,LATINO,LGBT,MARRIED,MUELLERAPP,NEC,NEC2,OVER45,OVER65,PARENTS,PARTY,PARTYBLACK,PARTYBYRACE,PARTYGENDER,PARTYID,PARTYSEX,PARTYWHITE,PHIL3,PTYIDEO7,QRACEAK,QRACEHI,QTYPE,RACE,RACE18,RACE18B,RACE2B,RACEAI,RACEFACTOR,REL5,RELIGN18,RELWHITE,RUSSIA18,SEXBYRACE,SEXUALHARASS,SEXUALHARASSB,SEXWED,SIZEPLAC,SIZEPLC3,SPANISHQ,STANUM,SUBURBWHITE,TAXCUTS18,TELEPOLL,TIME18HOU_1,TIME18HOUMONTH_1,TIME18HOUWEEK_1,TIME18HOU_2,TRACK,TRADE18,TRUMP2,TRUMP4,TRUMPFP18,TRUMPIMM,UNITED18,VERSION,VETVOTER,VOTE2016,VOTEISS18,WEIGHT,WHITEREL,WHNCLINC,WHTEVANG,WOMEN,WOMEN2,WPROTBRN,WPROTBRN3,HOU_Did not vote,HOU_Other,HOU_The Democratic candidate,HOU_The Republican candidate,SEN_Did not vote,SEN_Other,SEN_The Democratic candidate,SEN_The Republican candidate,GOV_Did not vote,GOV_Other,GOV_The Democratic candidate,GOV_The Republican candidate,ABORTION18_Keep it as is,ABORTION18_Omit,ABORTION18_Overturn it,AGE_18-29,AGE_30-44,AGE_45-65,AGE_65+,BOYSGIRLS_No children in home,"BOYSGIRLS_Yes, both boys and girls","BOYSGIRLS_Yes, only boys","BOYSGIRLS_Yes, only girls",BRNAGAIN_No,BRNAGAIN_Yes,PRECINCT_2,PRECINCT_3,PRECINCT_4,PRECINCT_5,PRECINCT_6,PRECINCT_7,PRECINCT_8,PRECINCT_9,PRECINCT_10,PRECINCT_11,PRECINCT_12,PRECINCT_13,PRECINCT_14,PRECINCT_16,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_44,PRECINCT_45,PRECINCT_46,PRECINCT_47,PRECINCT_48,PRECINCT_49,PRECINCT_50,PRECINCT_51,PRECINCT_52,PRECINCT_53,PRECINCT_54,PRECINCT_55,PRECINCT_56,PRECINCT_57,PRECINCT_58,PRECINCT_59,PRECINCT_60,PRECINCT_61,PRECINCT_62,PRECINCT_63,PRECINCT_64,PRECINCT_65,PRECINCT_67,PRECINCT_68,PRECINCT_70,PRECINCT_71,PRECINCT_72,PRECINCT_73,PRECINCT_74,PRECINCT_75,PRECINCT_76,PRECINCT_77,PRECINCT_78,PRECINCT_79,PRECINCT_80,PRECINCT_81,PRECINCT_82,PRECINCT_83,PRECINCT_84,PRECINCT_85,PRECINCT_86,PRECINCT_87,PRECINCT_88,PRECINCT_89,PRECINCT_90,PRECINCT_91,PRECINCT_92,PRECINCT_93,PRECINCT_94,PRECINCT_95,PRECINCT_96,PRECINCT_97,PRECINCT_98,PRECINCT_99,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_130,PRECINCT_131,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_160,PRECINCT_234,PRECINCT_,PRECINCT_1,PRECINCT_10,PRECINCT_100,PRECINCT_101,PRECINCT_102,PRECINCT_103,PRECINCT_104,PRECINCT_105,PRECINCT_106,PRECINCT_108,PRECINCT_109,PRECINCT_11,PRECINCT_110,PRECINCT_111,PRECINCT_112,PRECINCT_113,PRECINCT_114,PRECINCT_118,PRECINCT_119,PRECINCT_12,PRECINCT_122,PRECINCT_124,PRECINCT_125,PRECINCT_126,PRECINCT_127,PRECINCT_128,PRECINCT_13,PRECINCT_130,PRECINCT_133,PRECINCT_135,PRECINCT_136,PRECINCT_138,PRECINCT_141,PRECINCT_150,PRECINCT_152,PRECINCT_153,PRECINCT_156,PRECINCT_158,PRECINCT_16,PRECINCT_160,PRECINCT_17,PRECINCT_18,PRECINCT_19,PRECINCT_20,PRECINCT_21,PRECINCT_22,PRECINCT_23,PRECINCT_234,PRECINCT_24,PRECINCT_25,PRECINCT_26,PRECINCT_27,PRECINCT_28,PRECINCT_3,PRECINCT_30,PRECINCT_31,PRECINCT_32,PRECINCT_33,PRECINCT_34,PRECINCT_35,PRECINCT_36,PRECINCT_37,PRECINCT_38,PRECINCT_39,PRECINCT_4,PRECINCT_40,PRECINCT_41,PRECINCT_42,PRECINCT_43,PRECINCT_44,PRECINCT_45,PRECINCT_47,

In [1895]:
nat_df.shape

(19221, 415)

In [1383]:
nat_df["WOMEN"].value_counts()

0    14594
5     2288
4     1435
2      544
1      287
3       73
Name: WOMEN, dtype: int64

In [ ]:
# No answer = 0 
# Very Important = 5
# Somewhat Important = 4 
# Not too important = 2 
# Not at all important = 1 
# Omit = 3 

In [1346]:
# Omit VOTEISS18, VETVOTER, WHITEREL, WHTEVANG, WOMEN, WOMEN2, WPROTBRN, WPROTBRN3, ID,
# AGE8, AGE45, AGE60, AGE65, AGEBLACK, AGEBYRACE08, ATTEND16B, ATTREL, BACKSIDE Not complete
# BREAK18, BRNAGAIN, CALL, CHILDBOY, CHILDGIRL, EDUCCOLL, EDUCHS, EDUCWHITE, EDUCWHITEBYSEX,
# FAVDEM2, FAVREP2, FTVOTER18, GUNOWNER12, INC100k, INC30k, INC50k, INCOME3, 
# INCWHITE, LGBT, MARRIED, NEC2, QRACE3, PARTYBLACK, PARTYBLACK, PARTYSEX, PARTYWHITE, QTYPE
# QRACEAK, QRACEHI, RACE, RACE2B, RELWHITE, SEXUALHARASSB, SIZEPLC3, SPANISHQ, TELEPOLL, 
# TIME18HOU_1, TIME18HOUMONTH_1, TIME18HOUWEEK_1, TRUMP2,WHNCLINC, WHTEVANG, WOMEN2,WPROTBRN, 
# WPROTBRN3
# Later #VERSION, WEIGHTS

In [1365]:
#Do PARTYBYRACE PARTYSEX, RELIGN18, SUBURBWHITE

### Converting to a CSV for further cleaning 

In [1894]:
nat_df.to_csv("nat_df_uncleaned.csv", index = False)

In [1350]:
nat_df['BACKSIDE'].value_counts()
#remove those who did not complete the backside 

Complete        18622
Not complete      599
Name: BACKSIDE, dtype: int64